In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import heapq
import re
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\badru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\badru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
documents = [
'<p>Penuaan pada kulit wajah ditandai dengan munculnya keriput, vlek, kulit kering, dan pori-pori membesar. Hal ini terjadi pada wajah Anda? <em>Anti-Aging Solutions</em> dari Estetika dr.Affandi bisa jadi solusinya.</p>\r\n\r\n<ul>\r\n\t<li><strong>Rangkaian Anti-Aging Solutions</strong>\r\n\r\n\t<ul>\r\n\t\t<li><a href="https://klinikestetika.com/product">Krim Perawatan Anti-Aging*</a><br />\r\n\t\tJika digunakan secara teratur akan menyamarkan keriput halus dan menjaga kelembapan alami kulit.</li>\r\n\t\t<li>Treatments\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/ultherapy-by-ulthera-1">Ultherapy by Ulthera</a><br />\r\n\t\t\tSangat efektif untuk menyamarkan kerutan-kerutan dalam di wajah.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/laser-rejuvenation">Laser Rejuvenation</a>&nbsp;<br />\r\n\t\t\tMampu meremajakan kulit dan menghilangkan vlek-vlek hitam.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/botox-injection">Botox Injection</a></li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/filler-injection">Filler Injection</a></li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/premium-collagen-booster">Premium Collagen Booster</a></li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/threadlift">Threadlift / Tanam Benang</a></li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/skin-booster">Skin Booster / Ultrahydro Booste</a>r<br />\r\n\t\t\tmelembapkan kulit dari dalam sekaligus menjaga kekencangan &amp; elastisitas kulit.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/ipl-intense-pulsed-light">IPL (Intense Pulsed Light)</a><br />\r\n\t\t\tEfektif mengatasi noda kulit di wajah.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/autoroller">Autoroller</a><br />\r\n\t\t\tDapat merangsang produksi kolagen dan memperbaiki tekstur kulit wajah.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/prp">PRP Treatment</a><br />\r\n\t\t\tDapat menstimulasi produksi kolagen dan elastin pada kulit wajah.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/radio-frequency">Radio Frequency</a><br />\r\n\t\t\tMampu mengatasi masalah kerutan pada dahi, sekitar mata, dan garis senyum.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/bright-lift-aquapure">Bright Lift Aquapure</a><br />\r\n\t\t\tmenstimulasi peredaran darah di kulit sekaligus meregenerasi kolagen &amp; elastin.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/chemical-peeling-1">Chemical Peeling</a><br />\r\n\t\t\tDapat mengangkat lapisan kulit mati sehingga lapisan kulit baru terlihat.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/anti-aging-gold-facial">Anti Aging Gold Facial</a><br />\r\n\t\t\tMembantu meremajakan kulit &amp; mengurangi keriput / garis halus di wajah.<br />\r\n\t\t\t&nbsp;</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t</ul>\r\n\t</li>\r\n</ul>\r\n\r\n<p>*) harus dengan resep dokter</p>\r\n',
 '<p>Hiperpigmentasi dapat terjadi karena kulit sering terpapar sinar matahari, polusi, debu, dan berbagai unsur radikal bebas lainnya. Hal ini ditandai dengan munculnya bercak-bercak dan bintik-bintik hitam di kulit. Jika hiperpigmentasi kulit terjadi pada Anda, tenang saja, Estetika dr.Affandi punya solusinya.</p>\r\n\r\n<ul>\r\n\t<li><strong>Rangkaian Hyperpigmentation Solutions</strong>\r\n\r\n\t<ul>\r\n\t\t<li>\u200bProduk Perawatan*&nbsp;\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/product/detail/lotion-malam">Lotion Malam*</a><br />\r\n\t\t\tEfektif memudarkan vlek di wajah.</li>\r\n\t\t\t<li>\u200b\u200b<a href="https://klinikestetika.com/product/detail/krim-tirai-sinar-matahari-eksotika">Tirai Sinar Matahari Eksotika</a><br />\r\n\t\t\tdapat melindungi kulit wajah dari efek buruk sinar ultraviolet penyebab hiperpigmentasi.<br />\r\n\t\t\t&nbsp;</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t</ul>\r\n\t</li>\r\n\t<li>Treatments\r\n\t<ul>\r\n\t\t<li><a href="https://klinikestetika.com/services/all-treatments/laser-rejuvenation"><strong>Laser Rejuvenation</strong></a><br />\r\n\t\tmampu menghilangkan vlek-vlek hitam seperti <em>freckles, age-spots, lentigo, sun-burn spots dan melasma</em></li>\r\n\t\t<li><a href="https://klinikestetika.com/services/all-treatments/chemical-peeling-1"><strong>Chemical Peeling.</strong></a><br />\r\n\t\tjenis Deep Peeling efektif mengatasi hiperpigmentasi pada kulit Anda.</li>\r\n\t\t<li><a href="https://klinikestetika.com/services/all-treatments/prp"><strong>PRP Treatment</strong></a><br />\r\n\t\tdapat menstimulasi produksi kolagen dan elastin pada kulit wajah.</li>\r\n\t\t<li><a href="https://klinikestetika.com/services/all-treatments/brightening-infusion-and-injection"><strong>Brightening Injection &amp; Brightening Infusion</strong></a></li>\r\n\t\t<li><a href="https://klinikestetika.com/services/all-treatments/micropeel"><strong>Micropeel</strong></a><br />\r\n\t\tdapat mengangkat sel-sel kulit mati dan menstimulasi produksi kolagen yang akan memperbaiki tekstur kulit</li>\r\n\t</ul>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>*) harus dengan resep dokter</p>\r\n',
 '<p>Anda pasti ingin memiliki kulit wajah mulus tanpa bopeng, kan? Bopeng atau lubang bekas jerawat (acne scar) pasti sangat menggangu penampilan wajah Anda. Kini, Anda tak perlu khawatir. Estetika dr.Affandi memiliki <em>Scar Solutions</em> yang dapat mengatasi bopeng di wajah Anda.</p>\r\n\r\n<ul>\r\n\t<li><strong>Rangkaian Scar Solutions</strong>\r\n\r\n\t<ul>\r\n\t\t<li><a href="http://klinikestetika.com/product">\u200bKrim Perawatan*</a><br />\r\n\t\tbila digunakan secara rutin dapat menghindari munculnya masalah kulit penyebab scar pada wajah Anda</li>\r\n\t\t<li>Treatments\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/laser-eco2"><strong>Laser eCO2</strong></a><br />\r\n\t\t\tlaser eCO2 akan membantu luka mikroskopis kecil di atas sebagian kecil permukaan kulit. Cidera mikro ini merangsang remodeling dan produksi kolagen selama penyembuhan sehingga tekstur kulit menjadi lebih baik dan scar akan berkurang dengan maksimal (available : Semarang, Surabaya, Duren Tiga)</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/micropeel"><strong>Micropeel</strong></a><br />\r\n\t\t\tdapat mengangkat sel-sel kulit mati dan menstimulasi produksi kolagen yang akan memperbaiki tekstur kulit&nbsp;</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/prp"><strong>PRP Treatment</strong></a><br />\r\n\t\t\tmenstimulasi produksi kolagen dan elastin serta merangsang perbaikan sel kulit dengan pemanfaatan plasma darah&nbsp;</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/autoroller"><strong>Autoroller</strong></a><br />\r\n\t\t\tmerangsang produksi kolagen dan memperbaiki tekstur kulit</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t</ul>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>*) harus dengan resep dokter</p>\r\n',
 '<p>Kulit kusam akibat polusi dan sinar matahari merupakan masalah kulit yang sering dialami orang Indonesia. Apakah Anda juga mengalaminya? Kembalikan cerahnya kulitmu dengan <em>Brightening Solutions</em> dari Estetika dr.Affandi, yuk!</p>\r\n\r\n<ul>\r\n\t<li><strong>Rangkaian Brightening Solutions</strong>\r\n\r\n\t<ul>\r\n\t\t<li><a href="http://klinikestetika.com/product">Krim Perawatan*</a><br />\r\n\t\takan membuat kulit Anda tampak lebih cerah dan sehat dengan penggunaan rutin.</li>\r\n\t\t<li><a href="https://klinikestetika.com/product/detail/vitamin-pynocare-white">Vitamin Minum*</a><br />\r\n\t\tsuplemen untuk membuat kulit terlihat bersinar &amp; cerah.</li>\r\n\t\t<li>Treatments\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/laser-rejuvenation"><strong>Laser Rejuvenation</strong></a><br />\r\n\t\t\tefektif mencerahkan kulit yang kusam</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/skin-booster"><strong>Skin Booster / Ultrahydro Booster</strong></a></li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/bright-lift-aquapure"><strong>Bright Lift Aquapure for Rejuvenation</strong></a><br />\r\n\t\t\tmeremajakan &amp; mencerahkan kulit wajah</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/brightening-injection"><strong>Brightening Injection</strong></a><br />\r\n\t\t\tmengandung vitamin C untuk menangkal proses oksidasi penyebab kulit kusam</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/brightening-infusion-and-injection"><strong>Brightening Infusion</strong></a></li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/jet-peel"><strong>Jet Peel</strong></a><br />\r\n\t\t\tmembersihkan pori-pori yang tersumbat dan mengatasi bintik hitam di wajah</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/chemical-peeling-1"><strong>Chemical Peeling</strong></a><br />\r\n\t\t\tmampu mengelupaskan sel-sel kulit mati dan merangsang pembentukan sel kulit baru yang lebih cerah</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/iontophoresis"><strong>Iontophoresis</strong></a><br />\r\n\t\t\tmemanfaatkan Pro-Vitamin C untuk mencerahkan kulit</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/oxy-infusion"><strong>Oxy Infusion</strong></a><br />\r\n\t\t\tefektif untuk mencerahkan, menghaluskan, &amp; menutrisi kulit</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/advanced-whitening-facial"><strong>Advanced Whitening Facial</strong></a><br />\r\n\t\t\tdapat membersihkan sel kulit mati yang menyumbat pori-pori wajah dan mencerahkan kulit</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t</ul>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>*) harus dengan resep dokter</p>\r\n',
 '<p>Memiliki kontur wajah ideal merupakan idaman semua orang. Meskipun tidak semua orang terlahir dengan bentuk wajah yang ideal, kita tetap bisa memiliki kontur wajah sempurna dengan cara melakukan koreksi di bagian tertentu pada wajah, seperti: mempercantik bentuk bibir, memancungkan hidung, mengisi volume dagu hingga membuat wajah tampak lebih tirus, mengencangkan kulit wajah yang kendur, menghilangkan kerutan dan lain sebagainya.</p>\r\n\r\n<p>Melalui Estetika Contouring Solutions, Estetika dr. Affandi siap&nbsp; membantu mewujudkan kontur wajah ideal idaman sekaligus&nbsp; menjaga kesehatan kulit Anda.</p>\r\n\r\n<p><strong>Estetika Contouring Solutions Treatments</strong></p>\r\n\r\n<ul>\r\n\t<li><a href="https://klinikestetika.com/services/all-treatments/threadlift">Threadlift</a><br />\r\n\tTreatment dengan menanamkan benang di bawah jaringan kulit wajah ataupun anggota badan.</li>\r\n\t<li><a href="https://klinikestetika.com/services/all-treatments/botox-injection">Botox Injection</a><br />\r\n\tTreatment dengan menginjeksikan larutan Botox pada area kulit wajah atau area lain yang bermasalah</li>\r\n\t<li><a href="https://klinikestetika.com/services/all-treatments/ultherapy-by-ulthera-1">Ultherapy by Ulthera</a><br />\r\n\tTreatment untuk memperbaiki kontur wajah dengan menggunakan tekhnologi paling mutakhir yang memanfaatkan gelombang ultrasound</li>\r\n\t<li><a href="https://klinikestetika.com/services/all-treatments/filler-injection">Filler Injection</a><br />\r\n\tTreatment untuk mengisi lapisan di bawah kulit dengan cara menginjeksikan bahan alami (misal kolagen)</li>\r\n</ul>\r\n',
 '<p>Kata siapa remaja tidak perlu merawat kulitnya? Justru merawat kulit dari usia remaja sangat penting untuk dilakukan karena remaja bisa mengalami masalah kulit berminyak, berkomedo, dan berjerawat. Khusus para remaja, Estetika dr. Affandi menghadirkan Teen Skin Problem Solutions for Gen Z yang pas untuk mengatasi masalah kulit Anda.</p>\r\n\r\n<h3 style="color:#aaa; font-style:italic">&nbsp;</h3>\r\n\r\n<ul>\r\n\t<li><strong>Rangkaian Teen Skin Problem Solutions For Gen Z</strong>\r\n\r\n\t<ul>\r\n\t\t<li>Krim Perawatan\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/product/detail/krim-siang-malam-praktis">Krim Praktis*</a><br />\r\n\t\t\tKrim khusus untuk remaja ini mampu mengatasi masalah jerawat yang sering muncul di masa pubertas #RemajaEstetika</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t\t<li>Treatments\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/advanced-purifiying-facial">Advanced Purifying Facial</a><br />\r\n\t\t\tefektif menghilangkan &amp; membersihkan kotoran pada pori</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/acne-peeling">Acne Peeling</a></li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t</ul>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>*) harus dengan resep dokter</p>\r\n',
 '<p>Merawat Kesehatan rambut dan kulit kepala jugalah penting. Agar rambut dan kulit kepala Anda tetap sehat dan bebas dari kerontokan, ketombe, bahkan alopesia maka Estetika dr. Affandi memiliki <em>Hair Problem Solutions</em> untuk merawat dan mengatasi masalah kulit kepala Anda.</p>\r\n\r\n<ul>\r\n\t<li><strong>Rangkaian Hair Problem Solutions</strong>\r\n\r\n\t<ul>\r\n\t\t<li>\u200bKrim Perawatan*\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/product/detail/krim-rambut-rontok">Krim Perawatan Rambut Rontok*</a><br />\r\n\t\t\tEfektif mengatasi kerontokan rambut yang dapat berujung pada kebotakan (<em>Alopecia areata</em>) akibat kondisi folikel rambut dan kulit kepala yang tidak sehat.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/product/detail/hair-tonic-wanita">Hair Tonic*</a><br />\r\n\t\t\tEfektif berperan dalam pertumbuhan rambut dan mengatasi kerontokan rambut yang berat.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/product/detail/shampoo-rambut-rontok">Shampoo Rambut Rontok*</a><br />\r\n\t\t\tDapat mengatasi ketombe, psoriasis, dan eksim. Kulit kepala pun menjadi sehat dan rambut menjadi kuat, bercahaya, lembut, dan halus.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/product/detail/lotion-untuk-ketombe">Lotion untuk Ketombe*</a><br />\r\n\t\t\tDapat mengatasi kondisi ketombe dan rasa gatal yang berat di kulit Kepala (<em>Seborrheic dermatitis</em>)</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t\t<li>Vitamin Minum*\r\n\t\t<ul>\r\n\t\t\t<li>suplemen untuk menutrisi akar rambut dan merangsang pembentukan sel baru</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t\t<li>Treatments\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/autoroller">Autoroller</a><br />\r\n\t\t\tmerangsang folikel rambut untuk menumbuhkan rambut baru</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/prp">PRP Treatment</a></li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t</ul>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>*) harus dengan resep dokter</p>\r\n',
 '<p>Totok Wajah merupakan terapi yang sudah lama dilakukan di China dengan cara memberikan pijatan dengan teknik khusus di sekujur wajah dengan mengaktifkan titik-titik aura wajah. Tekanan itu membuat aliran darah di wajah menjadi lebih lancar sehingga wajah terlihat segar dan halus. Manfaat totok wajah :</p>\r\n\r\n<ol>\r\n\t<li>Mengencangkan wajah, memperlancar peredaran darah dan menghilangkan letih di area wajah serta punggung</li>\r\n\t<li>Memperkecil pori-pori sehingga wajah terlihat lebih bersih dan halus</li>\r\n\t<li>Mengurangi kantong mata dan dampak penuaan dini berupa kerutan</li>\r\n\t<li>Mengurangi flek atau noda hitam di wajah</li>\r\n</ol>\r\n\r\n<p>Metode ini diawali dengan pembersihan wajah lalu pijatan untuk membuat Anda rileks. Setelah itu barulah proses penotokan dimulai. Ada beberapa aturan yang perlu diperhatikan sebelum melakukan totok wajah. Pertama, Anda harus berusia 15 tahun ke atas , wajah tidak berjerawat , selang waktu antara totok pertama dan kedua adalah seminggu.</p>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p>Intimacy Care merupakan perawatan daerah kewanitaan dengan menggunakan ramuan yang berasal dari berbagai rempah yang beraroma harum semerbak khas tradisional keraton Jawa.</p>\r\n\r\n<p>Manfaat:</p>\r\n\r\n<ol>\r\n\t<li>Mengobati keputihan dan menghilangkan bau kurang sedap</li>\r\n\t<li>Sangat dianjurkan untuk wanita yang sudah berumah tangga karena dapat mengencangkan daerah intim wanita sehingga meningkatkan kualitas hubungan seksual</li>\r\n</ol>\r\n\r\n<p>Proses ini dilakukan dengan merebus rempah ratus sampai mendidih kemudian dituangkan di wadah yang diletakkan di bawah kursi khusus intimacy care. Duduklah dengan memakai kain penutup sehingga uap panasnya mengarah pada organ kewanitaan. Lakukan selama 30 menit. Selamat mencoba! (<strong>Time: 30 Minutes</strong>)</p>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p>Tanam benang atau <em>Threadlift</em> adalah prosedur kecantikan yang bertujuan untuk mengangkat dan membentuk bentuk wajah. Prosedur ini menggunakan benang khusus untuk menarik kulit kamu sehingga terlihat kencang.</p>\r\n\r\n<p>Treatment Threadlift menggunakan benang yang dapat diserap secara biologis oleh tubuh dan dapat merangsang pembentukan kolagen. Dengan kemajuan teknologi, benang yang digunakan saat ini memiliki kapasitas pengangkatan yang sangat baik, sehingga banyak digunakan untuk pasien yang menginginkan kesan &ldquo;lifting&rdquo; maksimal.&nbsp;</p>\r\n\r\n<p>&nbsp;&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melalukan Threadlift?</strong></p>\r\n\r\n<p><strong>&nbsp;</strong></p>\r\n\r\n<p>Perawatan Threadlift sangat cocok bagi Anda yang ingin terlihat muda kembali dan ingin mengatasi pengenduran alias tanda penuaan pada kulit, baik di wajah maupun bagian tubuh lainnya, seperti pinggul, lengan, paha, payudara, dll.</p>\r\n\r\n<p><strong>&nbsp;</strong></p>\r\n\r\n<p>Cara kerja dari T<a href="https://www.beautylogicaclinic.com/threadlift/" style="text-decoration:none;">hreadlift</a> adalah melakukan penanaman benang ke bаwаh jаrіngаn kulіt. Umumnya, prosedur ini diterapkan pada bagian wajah, misalnya seperti garis rаhаng, garis alis, аrеа bаwаh mata, dahi, pipi, dan bawah dagu.</p>\r\n\r\n<p>Bеnаng yang ditanamkan, nantinya akan mеnаrіk kulіt dari bаwаh, ѕеhіnggа ѕtruktur kulit уаng kеndur dapat dibuat menjadi lebih kencang. Penggunaan benang yang ditanamkan pada wajah, juga dapat berguna untuk merangsang pertumbuhan kolagen.</p>\r\n\r\n<p><strong>Manfaat Threadlift</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Meniruskan pipi,</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memancungkan hidung,</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi kerutan dan garis wajah,</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengencangkan kulit area lengan, payudara, pinggul, dan pantat</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi lemak dibawah dagu, serta merawat area tubuh lainnya.</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Berapa lama ketahanan efek Threadlift?</strong>&nbsp;</p>\r\n\r\n<p>Benang yang ditanamkan akan bertahan di wajah Anda selama kurang lebih 6-9 bulan. Setelah periode tersebut, benang akan menyerap ke tubuh secara alami.&nbsp;</p>\r\n\r\n<p>Meskipun benang sudah menyerap, kulit Anda akan tetap kencang hingga 12 bulan. Ini dikarenakan pembentukan kolagen dan serat elastin baru di bawah kulit.</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/A0fzDLFS84c">\r\n\r\n</iframe>',
 '<p>Tujuan: Membersihkan tubuh dari toxin, mengangkat sel-sel kulit mati, menghaluskan serta menjadikan kulit tampak lebih cerah.</p>\r\n\r\n<p>Menggunakan Essential Oil Fennel (Foeniculum vulgare), Rosemary (Rosemarinus officinalis), Lemon (Citrus limonom), Juniper (Juniperus communis) , dan Tyme (Thymus vulgaris ). Essential Oil yang terkandung didalamnya berfungsi untuk:</p>\r\n\r\n<ol>\r\n\t<li>Menurunkan obesitas</li>\r\n\t<li>Mengurangi nafsu makan</li>\r\n\t<li>Mempercepat pembakaran lemak</li>\r\n\t<li>Memperlancar peredaran darah</li>\r\n\t<li>Peremajaan kulit</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Time : 2 Hours 30 Minutes</strong></p>\r\n',
 '<p>Tujuan : Melebarkan pembuluh darah sehingga otot yang tegang menjadi relaks serta melembabkan kulit tubuh yang kering dan kusam.</p>\r\n\r\n<p>Menggunakan essential oil Mandarin, Cananga (Cananga macrophylla), Ylang ylang (Canangium odoratum), dan Lemon (Citrus limonom). Essential Oil yang terkandung didalamnya berfungsi untuk:</p>\r\n\r\n<ol>\r\n\t<li>Merawat kulit lelah menjadi relax</li>\r\n\t<li>Menumbuhkan selera humor dan kenyamanan secara menyeluruh</li>\r\n</ol>\r\n\r\n<p><strong>Time: 2 Hours</strong></p>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p>Tujuan : Bermanfaat untuk relaksasi tubuh, jiwa dan pikiran. Rasakan kesegaran baru setelah menjalani perawatan ini.</p>\r\n\r\n<p>Menggunakan essential oil Lavender (Lanvandula officinalis), Cananga (Cananga macrophylla), Patchouli (Pogostemon odoratum), dan Orange (Citrus aurantium). Essential Oil yang terkandung didalamnya berfungsi untuk :</p>\r\n\r\n<ol>\r\n\t<li>Menenangkan</li>\r\n\t<li>Melegakan pernafasan / flu</li>\r\n\t<li>Menurunkan tekanan darah</li>\r\n\t<li>Membantu mengatasi insomnia</li>\r\n</ol>\r\n\r\n<p><strong>Time: 2 Hours 30 Minutes</strong></p>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p>Tujuan : Bermanfaat untuk menghilangkan rasa letih dan pegal. Dengan rempah-rempah Bali yang harum dan hangat, Anda akan merasakan pemulihan energy setelah menjalani perawatan ini.</p>\r\n\r\n<p>Menggunakan essential oil Geranium (Pelargonium graveolens), Ylang ylang (Canangium odoratum), Patchouli (Pogostemon calolin), dan Grape fruit (Citrus paradisi). Essential Oil yang terkandung didalamnya berfungsi untuk :</p>\r\n\r\n<ol>\r\n\t<li>Menghilangkan rasa cemas, stress dan depresi</li>\r\n\t<li>Memberikan rasa aman, bahagia dan percaya diri</li>\r\n</ol>\r\n\r\n<p><strong>Time: 2 Hours</strong></p>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p>Persembahan untuk Anda yang Smart, Workaholic dan memerlukan konsentrasi tinggi. Perawatan ini mampu memberikan efek ketenangan sekaligus pengencangan pada kulit cantik Anda, menjadikan Anda senantiasa mempesona secantik prestasi yang akan Anda raih.</p>\r\n',
 '<p>Body Scrub adalah perawatan tubuh dengan menggunakan lulur yang sudah turun temurun dilakukan para putri raja dan permaisuri di kalangan kerajaan.<br />\r\nTujuan:</p>\r\n\r\n<ol>\r\n\t<li>Membersihkan kotoran yang melekat di tubuh</li>\r\n\t<li>Mengangkat sel-sel lapisan tanduk yang mati</li>\r\n\t<li>Membuka pori-pori sehingga kulit dapat bernapas</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<hr />\r\n<p>Body Mask adalah perawatan tubuh dengan menggunakan masker yang bersifat dingin dengan tujuan:</p>\r\n\r\n<ol>\r\n\t<li>Menyejukkan kulit</li>\r\n\t<li>Menjaga kehalusan dan kelembutan kulit</li>\r\n\t<li>Menipiskan flek-flek hitam di kulit</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>Variant:</p>\r\n\r\n<ol>\r\n\t<li>Scrub: Greentea, Mask: Greentea (Rp. 140.000,-, 1 Hour)</li>\r\n\t<li>Scrub: Chocolate, Mask: Chocolate ( Rp. 140.000,-, 1 Hour )</li>\r\n\t<li>Scrub: Lightening, Mask: Dead Sea Mud ( Rp. 140.000,-, 1 Hour )</li>\r\n\t<li>Scrub: Coffee, Mask: Seaweed ( Rp. 140.000,-, 1 Hour )</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p>Sebenarnya kecantikan bukan hanya untuk wajah, rambut, tubuh, tetapi KUKU juga wajib mendapatkan perhatian yang sama dengan bagian tubuh kita yang lain.</p>\r\n\r\n<p>Dengan demikian wajah yang bersih, wajah yang cantik, tubuh yang harum semerbak juga seharusnya dibarengi dengan penampilan kuku yang bersih, terawat rapi , cantik.</p>\r\n\r\n<p>Apa jadinya bila kita berpakaian rapi, rambut tertata rapi, wajah bermake up cantik tapi ketika bersalaman dengan orang tangan kita kasar, kuku kita kotor? Semua itu tidak akan terjadi kalau anda tahu cara merawatnya..</p>\r\n\r\n<p>Salah satu cara untuk mendapatkan kuku yang bersih, terawat, rapi dan cantik, kaki yang bersih, mulus, tidak pecah-pecah adalah dengan Manicure dan Pedicure.</p>\r\n\r\n<p>Dan seiring berkembangnya jaman syukurlah semakin banyak orang yang menyadari pentingnya merawat kuku baik kuku tangan , atau kuku kaki. Bagi orang yang tidak sempat merawat kuku tangan dan kaki, jangan khawatir karena saat ini banyak jasa manicure dan pedicure yang siap membantu merawat kuku tangan dan kaki anda.</p>\r\n\r\n<p>Tangan dan Kaki merupakan bagian tubuh yang tidak kalah penting, karena dengan ke dua tangan dan kaki semua kegiatan/ pekerjaan bisa terlaksana. Yang kadang-kadang membuat kaki dan tangan kita menjadi sangat lelah, dan akibatnya keadaan tangn, kaki , kuku tangan, kuku kaki jadi tidak sehat.</p>\r\n\r\n<p>Masalah-masalah yang sering terjadi pada kita atau sekitar kita misalnya kuku bengkak, kuku masuk, penebalan kutikula, kuku rapuh, kapalan, kaki pecah-pecah, kulit menebal.</p>\r\n\r\n<p>Manicure dan Pedicure merupakan cara yang tercepat untuk menangani masalah-masalah diatas.</p>\r\n\r\n<p>Perawatan khusus pada kuku tangan tidak hanya berguna untuk membuat keindahan kuku tetapi juga untuk menjaga kesehatan, kebersihan kaki dan tangan kita, dengan sedikit pemijatan dan perendaman tangan kita pun menjadi rileks.</p>\r\n',
 '<p><strong>Botulinum Toxin Injection</strong> adalah perawatan Anti-Wrinkle Injection atau biasa disebut <strong>Botox</strong>. <strong>Botulinum Toxin Injection</strong> digunakan untuk menghilangkan dan meminimalisir kerutan yang muncul pada kulit serta untuk melakukan koreksi di beberapa area tertentu pada wajah. Perawatan ini mengunakan toksin dari bakteri <em>Clostridium Botulinum</em>.</p>\r\n\r\n<p><strong>Botulinum Toxin Injection</strong> bekerja dengan cara memblokir aktivitas saraf dan otot, Inilah yang membuat <a href="https://www.alodokter.com/keriput" style="text-decoration:none;">keriput</a> atau kerutan di kulit sekitar lokasi penyuntikan tampak berkurang atau bahkan hilang serta mampu memperbaiki kontur wajah menjadi lebih estetis.</p>\r\n\r\n<p>Efek <strong>Botulinum Toxin Injection</strong> biasanya baru dirasakan beberapa hari setelah obat disuntikkan dan akan bertahan selama 3&ndash;6 bulan</p>\r\n\r\n<p>Manfaat perawatan<strong> Botulinum Toxin Injection</strong> :</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Menghilangkan kerutan/keriput di dahi, sudut mata dan antar alis</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menaikkan sudut bibir yang turun</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menaikkan alis</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghilangkan kantung mata</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Meniruskan rahang</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengecilkan cuping hidung</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi produksi keringat di ketiak dan telapak tangan</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>Keunggulan treatment <strong>Botulinum Toxin Injection</strong></p>\r\n\r\n<p>1. Pengerjaan cepat tidak butuh waktu lama</p>\r\n\r\n<p>2. Prosedur aman, tanpa operasi, dan rasa sakit</p>\r\n\r\n<p>3. Penyembuhan lebih cepat</p>\r\n\r\n<p>4. Setelah perawatan bisa langsung beraktivitas kembali</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Efek Samping&nbsp;Botulinum Toxin Injection</strong></p>\r\n\r\n<p><strong>Botulinum Toxin Injection</strong> sebagai salah satu prosedur yang resmi dan sudah di setujui oleh<a href="https://www.fda.gov/home" style="text-decoration:none;"> Food and Drug Administration (FDA)</a> . Peresmian dan surat izin <strong>Botulinum Toxin Injection</strong> sudah terbit sejak 1991. Semenjak saat itu <strong>Botulinum Toxin Injection</strong> pun menjadi pilihan bagi 6,7 juta perempuan di AS untuk menghilangkan kerutan.</p>\r\n\r\n<p>Kalaupun ada efek samping suntikan pada bagian tubuh (wajah), efek yang timbul hanyalah bersifat sementara.</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/AUfWgfVFHnk">\r\n\r\n</iframe>',
 '<p>Tangan merupakan bagian tubuh yang cukup mendapat perhatian lawan bicara kita, oleh karena itu rawatlah agar kelihatan cantik, segar dan sehat. Tangan pula yang terus dipakai selama melakukan kegiatan. Dapat dibayangkan betapa lelahnya tangan manusia. Bagian ini juga cenderung kering karena seringnya dicuci untuk menjaga kebersiahan dan kesehatan.<br />\r\nTujuan:</p>\r\n\r\n<ol>\r\n\t<li>Merelaksasi otot tangan</li>\r\n\t<li>Mengangkat sel kulit mati</li>\r\n\t<li>Menjaga kelembapan kulit tangan</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>Perawatan ini dimulai dengan merendam tangan dalam waskom dan kuku tangan disikat dengan bath gel. Seluruh tangan dilap dengan air hangat menggunakan washlap. Scrub dengan papaya enzim,bersihkan dengan air hangat,pijat tangan dengan massage oil, bilas dengan air hangat. Kemudian gunakan paraffin,tunggu beberapa saat. Setelah selesai,bilas tangan dengan air hangat kembali, oleskan body lotion, dan semprotkan telapak tangan dengan handspray. <strong>Time: 30 Minutes</strong></p>\r\n',
 '<p>Kaki merupakan bagian tubuh yang mempunyai tugas berat yaitu harus menopang beban tubuh selama kita beraktifitas. Dapat dibayangkan betapa lelahnya kaki manusia. Bagian ini juga cenderung kering karena terpaan panas maupun dingin, seringnya dicuci untuk menjaga kebersihan dan kesehatan.</p>\r\n\r\n<p>Tujuan sebenarnya hampir sama dengan hand spa with paraffin yakni untuk merelaksasi otot kaki dan menjaga kelembapan kulit kaki.</p>\r\n\r\n<p>Perawatan ini dimulai dengan merendam kaki dalam waskom dan kuku kaki disikat dengan bath gel. Seluruh kaki dilap dengan air hangat menggunakan washlap. Kemudian tumit dibersihkan dengan penghalus tumit. Rendam kaki dalam footbath selama 5 menit. Kaki sampai lutut discrub dengan foot scrub. Bersihkan dengan air hangat. Stretching dengan minyak selama 15 menit. Pijat kaki, bilas dengan air hangat. Oleskan masker kaki. Bungkus kaki dengan handuk hangat. Oleskan lotion for leg. Kemudian terkahir telapak kaki disemprot dengan foot spray. Voila ! Kaki siap kembali diajak untuk beraktifitas. <strong>Time: 30 Minutes</strong></p>\r\n',
 '<p>Massage atau pemijatan berguna untuk kesehatan dan kecantikan. Dalam bahasa Arab artinya adalah tekanan lembut, sedangkan dalam bahasa Yunani berarti meremas. Pada tahun 5000 SM di Cina, seni massage ini sudah dikembangkan dan terus meluas ke Persia, Jepang dan Mesir Purba.</p>\r\n\r\n<p>Manfaat:</p>\r\n\r\n<ol>\r\n\t<li>Mengurangi ketegangan dan merelaksasi otot tubuh</li>\r\n\t<li>Melancarkan sirkulasi darah dan mengurangi nyeri</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>Aliran hangat dari tubuh therapist yang secara berkesinambungan masuk ke dalam tubuh pasien akan terasa nyaman karena proses pemijatan yang bersifat body contact. Arah pemijatan pun selalu menuju jantung dengan tujuan menstimulasi darah didalam tubuh.</p>\r\n\r\n<p>Jenis:</p>\r\n\r\n<ol>\r\n\t<li>Javanese Massage, pemijatan ini menggunakan tekanan yang mantap</li>\r\n\t<li>French Massage, pemijatan ini menggunakan penekanan dengan kekuatan sedang, dan banyak gerakan-gerakan yang berguna untuk menghilangkan selulite.</li>\r\n\t<li>Swedish Massage, pemijatan ini menggunakan tekanan yang lembut</li>\r\n\t<li>Combination Massage, pemijatan ini menggunakan kombinasi dari ketiga teknik pemijatan diatas.</li>\r\n</ol>\r\n\r\n<p><strong>Time: 1 Hour or 1 Hour 30 Minutes</strong></p>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p>Waxing adalah metode semi permanen yang dapat menghilangkan bulu halus dari akar. Bulu halus baru, tidak akan tumbuh kembali di daerah yang diwaxing selama 2-4 minggu. Ada banyak jenis waxing yang cocok untuk menghilangkan bulu halus yang tidak diinginkan. Salah satunya adalah Sugar Waxing yang diklaim paling aman untuk dilakukan karena bebas dari bahan kimia.</p>\r\n\r\n<p>Sugar Waxing dilakukan dengan mengoleskan campuran gula dan madu yang lebih dikenal dengan istilah caramel ke permukaan kulit, kemudian kain tipis dibentangkan di area yang hendak diwaxing, ditekan dan ditarik dengan gerakan cepat supaya bulu halus ikut tertarik bersama dengan kain tersebut. Setelah waxing, dioleskan krim antibiotik topikal dan krim hidrokortison 1% sebagai anti peradangan ke daerah yang telah di waxing.</p>\r\n\r\n<p>Variant:</p>\r\n\r\n<ol>\r\n\t<li>Under Arms</li>\r\n\t<li>Arms</li>\r\n\t<li>Full Legs</li>\r\n\t<li>Half Legs</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p>Saat pernikahan sudah di depan mata, tentunya berbagai persiapan menjelangnya sudah semakin matang. Namun, bagaimana dengan tubuh Anda? Membagi perhatian terhadap kesiapan fisik kerapkali terabaikan karena sibuk dengan berbagai kebutuhan yang dianggap lebih besar. Jangan sepelekan perawatan tubuh pranikah karena manfaatnya berlipat, untuk merawat kecantikan kulit, kesehatan fisik, membantu perampingan tubuh, hingga merawat vitalitas organ intim perempuan. Pre Wedding Treatment merupakan sebuah rangkaian perawatan tubuh yang menyeluruh bagi seseorang yang hendak menikah, agar si mempelai dapat tampil segar dan memukau di hadapan pasangannya ketika menikah.</p>\r\n\r\n<p><strong>Kapan waktunya melakukan perawatan ?</strong><br />\r\nDisarankan perawatan pre wedding dilakukan enam bulan sebelum hari pernikahan. Pilihan lainnya tiga bulan sebelum hari H agar hasilnya optimal untuk tubuh. Jika pun waktunya terbatas, perawatan bisa dilakukan satu bulan sebelum hari H dan masih bisa dilanjutkan setelah hari pernikahan jika masih ada paket perawatan tersisa. Satu kali perawatan memakan waktu sekitar 3-4 jam. Jadi, tunggu apa lagi, pilih perawatan tubuh sesuai kebutuhan dan manjakan diri menjelang hari pernikahan, selain juga untuk menunjang kesehatan tubuh.</p>\r\n\r\n<p><strong>A. 1x Kunjungan (3 Hours 30 Minutes)</strong><br />\r\nUrutan Treatment :</p>\r\n\r\n<ol>\r\n\t<li>Massage and Pelvic Massage</li>\r\n\t<li>Scrub: Ligthening Scrub</li>\r\n\t<li>Masker: Dead Sea Mud</li>\r\n\t<li>Heating blanket + wraping / Sauna / Steam</li>\r\n\t<li>Hydrobath: Foaming Milkbath and Bath Gel</li>\r\n\t<li>Bust Treatment (Perawatan Payudara) dan Intimacy Care (Perawatan daerah Kewanitaan)</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<hr />\r\n<p><strong>B. 6 X KUNJUNGAN</strong></p>\r\n\r\n<ol>\r\n\t<li><strong>Kunjungan 1 (2 Hours 30 Minutes)</strong><br />\r\n\tUrutan Treatment :\r\n\t<ul>\r\n\t\t<li>Massage and Pelvic Massage</li>\r\n\t\t<li>Scrub: Greentea Scrub</li>\r\n\t\t<li>Masker: Greentea</li>\r\n\t\t<li>Heating blanket + wraping / Sauna / Steam</li>\r\n\t\t<li>Hydrobath: Foaming Milkbath and Bath Gel</li>\r\n\t\t<li>Bust Treatment</li>\r\n\t\t<li>Intimacy Care</li>\r\n\t</ul>\r\n\t</li>\r\n\t<li><strong>Kunjungan 2 (3 Hours)</strong><br />\r\n\tUrutan Treatment :\r\n\t<ul>\r\n\t\t<li>Massage and Pelvic Massage</li>\r\n\t\t<li>Scrub: Ligthening Scrub</li>\r\n\t\t<li>Masker: Dead Sea Mud</li>\r\n\t\t<li>Heating blanket + wraping / Sauna / Steam</li>\r\n\t\t<li>Hydrobath: Foaming Milkbath and Bath Gel</li>\r\n\t\t<li>Bust Treatment</li>\r\n\t\t<li>Intimacy Care</li>\r\n\t</ul>\r\n\t</li>\r\n\t<li><strong>Kunjungan 3 (3 Hours)</strong><br />\r\n\tUrutan Treatment :\r\n\t<ul>\r\n\t\t<li>Massage and Pelvic Massage</li>\r\n\t\t<li>Scrub: Chocolate Scrub</li>\r\n\t\t<li>Masker: Chocolate</li>\r\n\t\t<li>Heating blanket + wraping / Sauna / Steam</li>\r\n\t\t<li>Hydrobath: Foaming Milkbath and Bath Gel</li>\r\n\t\t<li>Bust Treatment</li>\r\n\t\t<li>Intimacy Care</li>\r\n\t</ul>\r\n\t</li>\r\n\t<li><strong>Kunjungan 4 (3 Hours)</strong><br />\r\n\tUrutan Treatment :\r\n\t<ul>\r\n\t\t<li>Massage and Pelvic Massage</li>\r\n\t\t<li>Scrub: Coffee Scrub</li>\r\n\t\t<li>Masker: Seaweed</li>\r\n\t\t<li>Heating blanket + wraping / Sauna / Steam</li>\r\n\t\t<li>Hydrobath: Foaming Milkbath and Bath Gel</li>\r\n\t\t<li>Bust Treatment</li>\r\n\t\t<li>Intimacy Care and Totok Wajah</li>\r\n\t</ul>\r\n\t</li>\r\n\t<li><strong>Kunjungan 5 (3 Hours)</strong><br />\r\n\tUrutan Treatment :\r\n\t<ul>\r\n\t\t<li>Massage and Pelvic Massage</li>\r\n\t\t<li>Scrub: Murut Scrub</li>\r\n\t\t<li>Masker: Boreh</li>\r\n\t\t<li>Heating blanket + wraping / Sauna / Steam</li>\r\n\t\t<li>Hydrobath: Foaming Milkbath and Bath Gel</li>\r\n\t\t<li>Bust Treatment</li>\r\n\t\t<li>Intimacy Care</li>\r\n\t\t<li>Totok Wajah</li>\r\n\t</ul>\r\n\t</li>\r\n\t<li><strong>Kunjungan 6 (4 Hours)</strong><br />\r\n\tUrutan Treatment :\r\n\t<ul>\r\n\t\t<li>Massage and Pelvic Massage</li>\r\n\t\t<li>Scrub:</li>\r\n\t\t<li>Masker:</li>\r\n\t\t<li>Heating blanket + wraping / Sauna / Steam</li>\r\n\t\t<li>Hydrobath: Foaming Milkbath and Bath Gel</li>\r\n\t\t<li>Bust Treatment and Intimacy Care</li>\r\n\t\t<li>Totok Wajah</li>\r\n\t\t<li>Facial</li>\r\n\t</ul>\r\n\t</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n',
 '<p><strong>Apa itu Filler Injection?</strong></p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>Perawatan Filler adalah proses penyuntikan cairan hyaluronic acid ke dalam lapisan kulit. Manfaatnya bisa digunakan untuk memberi volume, menyamarkan garis dan kerutan pada wajah sehingga tampak muda.</p>\r\n\r\n<p>filler bekerja dengan mengisi bagian kulit wajah untuk mengembalikan volume dan kekencangan kulit wajah.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Filler Injection</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengisi kerutan wajah yang dalam, seperti garis senyum, dan area bibir</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membentuk tulang pipi lebih tinggi dan berisi</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membuat dagu lebih lancip</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengisi pipi yang kempot</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengisi bagian belakang hidung sehingga lebih mancung</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengisi kantong mata yang cekung</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Filler Injection?</strong></p>\r\n\r\n<p>Filler Injection dilakukan dengan cara menyuntikan bagian wajah seperti pipi, hidung, bibir, dagu, rahang, dan lainnya dengan cairan seperti asam hialuronat, kolagen, atau zat sintetis ke dalam lapisan kulit.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Berapa lama ketahanan efek dari Filler Injection?</strong></p>\r\n\r\n<p>&nbsp;Setiap jenis filler mengandung bahan dan harga yang berbeda sehingga ketahanannya pun juga berbeda. Umumnya, semakin mahal harga filler maka semakin tahan pula efeknya.&nbsp;</p>\r\n\r\n<p>&nbsp;Oleh karena itu, Anda perlu berkonsultasi dengan dokter sebelum melakukan treatment ini.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah ada efek samping dari Filler Injection?</strong></p>\r\n\r\n<p>&nbsp;Filler Injection memang tidak menimbulkan efek samping alergi, penolakan benda asing, dan peradangan dari tubuh.</p>\r\n\r\n<p>&nbsp;Namun, beberapa orang dapat memunculkan gejala yang bersifat normal dan akan hilang dengan sendirinya, seperti kemerahan, bengkak, dan gatal.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/ByeeG5AR92A\r\n">\r\n</iframe>\r\n',
 '<p><strong>Apa itu Jet Peel?</strong></p>\r\n\r\n<p>Jet Peel adalah perawatan kulit dengan teknik modern menggunakan alat khusus yang memberikan tekanan oksigen tinggi dan cairan khusus berupa Hyaluronic Acid pada wajah yang bertujuan untuk mengangkat sel kulit mati dengan proses deep cleansing secara optimal.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Jet Peel</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Melembabkan dan mencerahkan kulit wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengencangkan pori-pori&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menyamarkan flek hitam dan noda bekas jerawat</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengontrol produksi minyak berlebih&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menyamarkan garis-garis halus dan keriput di wajah</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Jet Peel?</strong></p>\r\n\r\n<p>Treatment Jet Peel merupakan solusi bagi Anda yang ingin mengatasi masalah noda bekas jerawat dan juga mengangkat sel kulit mati pada wajah agar regenerasi kulit lebih optimal.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Jet Peel?</strong></p>\r\n\r\n<p>Pada tahap awal Jeet Peel, dokter akan menggunakan alat berteknologi modern yang akan memberikan tekanan oksigen tinggi dan cairan khusus ke wajah berupa serum eksfoliator yang kemudian dilanjutkan dengan serum Hyaluronic Acid yang akan dialirkan ke kulit. Cairan serum tersebut itu dialirkan dengan kecepatan 200 meter per detik dan serum akan masuk ke lapisan kulit secara maksimal.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Efek Samping Jet Peel</strong></p>\r\n\r\n<p>Efek samping yang ditimbulkan dari treatment ini adalah timbul kemerahan pada wajah. Namun efek samping dari treatment ini tidak akan lama dan bersifat sementara dapat hilang dalam beberapa hari setelah treatment Jeet Peel.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/ZF1s3Xpe8oU\r\n">\r\n</iframe>\r\n\r\n',
 '<p>Jerawat memang menjadi masalah kulit yang sering dialami oleh banyak orang. Akan tetapi, Estetika dr. Affandi memiliki <em>Acne Solutions</em> yang bisa mengatasi jerawat di wajah Anda. So, Anda tidak perlu khawatir</p>\r\n\r\n<ul>\r\n\t<li><strong>Rangkaian Acne Solutions</strong>\r\n\r\n\t<ul>\r\n\t\t<li>Krim Perawatan\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/product/detail/krim-pagi">Krim Pagi*</a><br />\r\n\t\t\tuntuk mengobati jerawat, meremajakan kulit, menyamarkan keriput halus, dan menjaga kelembapan alami kulit.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/product/detail/krim-malam-i">Krim Malam*</a><br />\r\n\t\t\tdapat mengatasi noda-noda di wajah akibat bekas jerawat maupun paparan sinar matahari.</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t\t<li>Obat Minum Untuk Jerawat*<br />\r\n\t\tAkan membantu mempercepat proses penyembuhan jerawat pada kulit wajah Anda.</li>\r\n\t\t<li>Treatments\r\n\t\t<ul>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/injeksi-jerawat"><strong>Injeksi Jerawat</strong></a><br />\r\n\t\t\tMerupakan perawatan tepat untuk mengatasi jerawat meradang. Jerawat yang bernanah akan dikeluarkan terlebih dahulu, setelahnya jerawat akan diinjeksi.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/acne-peeling"><strong>Acne Peeling</strong></a><br />\r\n\t\t\tMenjadi salah satu alternatif terbaik untuk menghilangkan jerawat dan bekasnya.</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/ipl-intense-pulsed-light"><strong>IPL (Intense Pulsed Light)</strong></a><br />\r\n\t\t\tefektif mengeringkan jerawat yang meradang</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/bright-lift-aquapure-for-oily-and-acne-skin"><strong>Bright Lift Aquapure for Acne / Oily Skin</strong></a><br />\r\n\t\t\tefektif mengangkat sel kulit mati &amp; kotoran lain di permukaan kulit (<em>deep cleansing</em>)</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/advanced-purifiying-facial"><strong>Advanced Purifying Facial</strong></a><br />\r\n\t\t\tefektif untuk menghilangkan &amp; membersihkan kotoran pada pori</li>\r\n\t\t\t<li><a href="https://klinikestetika.com/services/all-treatments/advanced-detox-deep-pore-cleanse"><strong>Detox &amp; Deep Pore Cleans Facial</strong></a><br />\r\n\t\t\tmembersihkan sel kulit mati &amp; menyerap kelebihan minyak di kulit</li>\r\n\t\t</ul>\r\n\t\t</li>\r\n\t</ul>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>*) harus dengan resep dokter</p>\r\n',
 '<p><strong>Apa itu Ultheraphy by Ulthera?</strong></p>\r\n\r\n<p>Ultheraphy adalah salah satu treatment non-invasif (tanpa operasi) yang menggunakan mesin dengan teknologi paling mutakhir yang memanfaatkan gelombang ultrasound. Treatment Ultheraphy berfungsi untuk membentuk kolagen baru pada lapisan kulit serta mampu melakukan koreksi bentuk beberapa area pada wajah (face contouring).</p>\r\n\r\n<p>Dengan kecanggihan mesin Ulthera produksi Amerika, ultrasound akan menghasilkan sinar yang dapat merangsang pembentukan kolagen, memperindah/menyeimbangkan bentuk tertentu pada wajah tanpa merusak dan berbahaya bagi permukaan kulit.</p>\r\n\r\n<p><strong>Manfaat Ultheraphy</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Memperbaiki tesktur dan elastisitas kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>mengatasi kerutan di area wajah, leher, serta dada</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membentuk V shape contour pada wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Bentuk dagu menjadi lebih slim</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membuat tampilan tulang pipi menjadi lebih sempurna</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membuat garis rahang menjadi lebih tegas pada pria</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>Treatment ini sangat efektif bagi Anda yang ingin melakukan koreksi bentuk tertentu pada area wajah ataupun bagi Anda yang mengalami proses penuaan pada kulit baik karena faktor usia maupun terpapar penyebab penuaan. Bentu wajah Anda akan mendekati kontur wajah yang ideal dan kulit Anda akan kembali mulus serta kembali kencang seperti remaja.</p>\r\n\r\n<p><strong>Keunggulan melakukan treatment Ultheraphy</strong></p>\r\n\r\n<p><strong>Prosedur non-invasif (tanpa operasi)</strong></p>\r\n\r\n<p>Ultheraphy merupakan pengobatan kulit yang tidak melalui proses operasi seperti menggunakan jahitan dan suntikan.</p>\r\n\r\n<p>Selain itu, Ultheraphy juga satu-satunya pengobatan yang terbukti lolos uji Food and Drug Administration (FDA) atau Badan Pengawas Obat dan Makanan Amerika Serikat.</p>\r\n\r\n<p><strong>Tidak sakit</strong></p>\r\n\r\n<p>Pasien akan diterapi selama kurang lebih 45 menit. Anda hanya merasa kulit akan hangat karena sinar ultrasound dan seperti ada jarum kecil yang bergerak di kulit tanpa rasa sakit.</p>\r\n\r\n<p><strong>Hasil yang memuaskan dan bertahan bertahun-tahun</strong></p>\r\n\r\n<p>Cukup dengan beberapa kali treatment Ultheraphy, Anda akan merasakan hasil yang bertahan satu hingga dua tahun lamanya.</p>\r\n\r\n<p>Agar hasil yang maksimal, Ultheraphy ini harus diimbangi dengan gaya hidup sehat dari pasien.</p>\r\n\r\n<p><strong>Tanpa efek samping</strong></p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/ZyW-sLqhYpM">\r\n</iframe>\r\n',
 '<p><strong>Apa itu Laser Rejuvenation?</strong></p>\r\n\r\n<p>Laser rejuvenation adalah prosedur perawatan dengan melapisi ulang kulit yang memanfaatkan laser. Perawatan ini bertujuan memperbaiki penampilan kulit yang rusak akibat beragam masalah kulit, seperti:</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>kerusakan kulit akibat sinar matahari</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>bekas jerawat</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>cacar air</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>face scar</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>Prosedur ini pun diklaim dapat meningkatkan penampilan kulit, yaitu mengurangi kemunculan garis halus di sekitar mulut, mata, dan pipi, mengobati hilangnya warna kulit, serta mengatasi hiperpigmentasi kulit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Cara kerja&nbsp;</strong></p>\r\n\r\n<p>Pada hari perawatan dimulai, dokter akan mengoleskan anestesi topikal (bius lokal) pada area yang mengalami kerusakan, hal ini bertujuan mengurangi rasa sakit. Lalu, kulit akan dibersihkan untuk mengangkat minyak berlebih, kotoran, dan bakteri.</p>\r\n\r\n<p>Setelah itu, dokter akan mulai dengan menggunakan laser berdasarkan metode yang Anda pilih. Laser akan digerakkan perlahan di sekitar area kulit yang ditentukan.</p>\r\n\r\n<p>Bila sudah selesai, dokter membalut daerah tersebut guna melindungi bagian kulit pada akhir prosedur.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Perawatan setelah prosedur</strong></p>\r\n\r\n<p>Setelah prosedur laser rejuvenation selesai, Anda mungkin akan merasakan panas atau iritasi Tak perlu khawatir karena akan diberikan krim Post-Laser dan digunakan selama 3 hari.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/NjOmfdh7uRs">\r\n</iframe>\r\n',
 '<p><strong>Apa itu PRP?</strong></p>\r\n\r\n<p>Platelet Rich Plasma (PRP) merupakan terapi yang menggunakan darah pasien sendiri yakni bagian plasma yang kaya akan platelet dan dimasukan kembali ke dalam kulit untuk memicu proses pembentukan kolagen dan penyembuhan</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat treatment PRP&nbsp;</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengatasi kerontokan rambut</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Sebagai skin rejuvenation&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengatasi scar bekas jerawat pada wajah&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi stretch mark</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Cara kerja treatment PRP&nbsp;</strong></p>\r\n\r\n<p>Pada tahap pertama treatment PRP, pasien akan diambil darahnya sebanyak 20-60 mililiter. Setelah itu, darah tersebut akan dimasukkan ke dalam alat centrifuge (alat pemutar) yang berfungsi untuk memisahkan antara sel darah dan serum plasma. Serum yang dihasilkan mengandung dua bagian; salah satu dari Platelet Poor Plasma dan yang lainnya dari Platelet Rich Plasma (atau disingkat PRP). Serum PRP dapat digunakan dengan dua cara, yaitu dengan menyuntikkan serum ke dalam kulit atau dengan menggunakan alat micro-needle untuk membuat channel mikro untuk memasukkan PRP ke dalam kulit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah treatment PRP menimbulkan efek samping?</strong><strong>&nbsp;</strong></p>\r\n\r\n<p>Platelet Rich Plasma adalah metode yang cocok untuk siapa saja karena memberikan hasil yang sangat baik dan alami. Dapat digunakan pada semua jenis kulit, bekerja cepat, dan memberikan hasil yang bagus dengan waktu penyembuhan yang minimal. Micro-needling dapat membuat wajah merah sementara dengan downtime sekitar 2-3 hari.</p>\r\n\r\n<p>Untuk daerah yang mendapatkan injeksi Akan timbul rasa sakit ringan, Namun, efek samping ini hanya bertahan beberapa hari.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/SPtCwyItw1A">\r\n</iframe>\r\n',
 '<p><strong>Apa itu Autoroller?</strong></p>\r\n\r\n<p>Autoroller adalah perawatan kulit dengan membuat luka yang dimasukkan zat aktif sehingga terjadi regenerasi kulit secara alami.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Autoroller</strong></p>\r\n\r\n<ul>\r\n\t<li>Mengurangi kerutan dan garis wajah</li>\r\n\t<li>Mengatasi masalah stretch mark</li>\r\n\t<li>Merangsang pertumbuhan rambut dan mencegah kerontokan</li>\r\n\t<li>Memperbaiki tekstur kulit (bopeng)</li>\r\n\t<li>Merangsang produksi kolagen</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Autoroller?</strong></p>\r\n\r\n<p>Treatment Autoroller cocok bagi Anda yang ingin menghilangkan scar/ bekas luka, kerutan, dan lain lain.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Varian dari Autoroller</strong></p>\r\n\r\n<ul>\r\n\t<li>Autoroller Anti-Aging</li>\r\n\t<li>Autoroller Lifting</li>\r\n\t<li>Autoroller Whitening</li>\r\n\t<li>Autoroller Scar</li>\r\n\t<li>Autoroller Brightening</li>\r\n\t<li>Autoroller Glowing</li>\r\n\t<li>Autoroller for Oily Skin</li>\r\n\t<li>Autoroller Alopecia</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/ROZYtxaxGig\r\n">\r\n</iframe>\r\n',
 '<p><strong>Apa itu Skin Booster?</strong></p>\r\n\r\n<p>Skin Booster adalah konsep perawatan kulit yang dilakukan dengan menyuntikkan hyaluronic acid (HA) atau asam hialuronat dosis rendah ke lapisan kulit bagian atas (dermis).&nbsp;</p>\r\n\r\n<p><em>Hyaluronic acid </em>&nbsp;merupakan zat yang secara natural terdapat pada kulit manusia, terutama pada jaringan di sekitar mata. Semakin tinggi kandungan HA di dalam kulit, semakin kenyal dan lembap kulit tersebut sehingga tidak rentan mengalami kerutan maupun <a href="https://www.sehatq.com/artikel/penuaan-dini-ciri-ciri-penyebab-dan-cara-mencegahnya" style="text-decoration:none;">tanda-tanda penuaan</a> lainnya.</p>\r\n\r\n<p><strong>Manfaat Skin Booster</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Menjadikan kulit sehat dan cerah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Kulit lebih kencang</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Kulit lebih kenyal dan lembab</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menutrisi kulit wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memperbaiki jaringan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Merangsang pertumbuhan kolagen pada kulit&nbsp;</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan skin booster?</strong></p>\r\n\r\n<p>Treatment Skin Booster merupakan solusi bagi Anda yang ingin mengatasi masalah penuaan dan menjaga kelembapan pada kulit. Treatment ini dapat dilakukan pada semua usia baik tua maupun muda.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja skin booster?</strong></p>\r\n\r\n<p>Pada tahap pertama Treatment Skin Booster, dokter akan melakukan anastesi pada wajah pasien. Kemudian Skin Booster disuntikkan pada titik-titik tertentu wajah pasien dengan jarum yang sangat kecil.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Efek samping Skin Booster</strong></p>\r\n\r\n<p>Efek samping yang ditimbulkan dari treatment ini dapat seperti pembengkakan dan kemerahan pada titik-titik area injeksi. Namun, efek samping tersebut hanya sementara dan hilang dalam beberapa hari setelah treatment Skin Booster.</p>\r\n',
 '<p><strong>Apa itu IPL?</strong></p>\r\n\r\n<p>Treatment IPL atau <em>Intense Pulsed Light</em> adalah sebuah prosedur untuk meningkatkan tampilan warna dan tekstur kulit menggunakan gelombang sinar, hampir seperti perawatan laser. Prosedur ini dapat membantu menghilangkan kerutan, bintik-bintik, bercak merah, dan rambut yang tidak diinginkan di area kulit.</p>\r\n\r\n<p>IPL treatment menggunakan berbagai macam gelombang cahaya yang dipadukan agar dapat menargetkan area yang lebih besar. Bahkan, treatment IPL juga memiliki efek samping lebih sedikit dibandingkan dengan treatment laser.&nbsp;</p>\r\n\r\n<p><strong>Manfaat IPL</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Menghilangkan kerutan dan garis halus&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghilangkan noda hitam</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menyamarkan bekas luka</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Merontokkan bulu</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengatasi jerawat</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p><strong>Apakah saya dapat melakukan IPL?</strong></p>\r\n\r\n<p>Treatment IPL menjadi solusi terbaik non-invasif (tanpa operasi) bagi Anda yang ingin memperbaiki kerusakan pada kulit baik yang disebabkan karena paparan sinar matahari maupun kondisi kesehatan tertentu.</p>\r\n\r\n<p><strong>Prosedur IPL terdiri dari proses berikut:</strong></p>\r\n\r\n<p>&middot; Pemberian bius topikal yang dioleskan pada area IPL</p>\r\n\r\n<p>&middot; Pemberian gel dingin di area target. Perangkat IPL umumnya telah memiliki sistem pendingin terintegrasi.</p>\r\n\r\n<p>&middot; Selanjutnya permukaan kaca halus dari alat IPL akan diaplikasikan pada kulit. Perangkat ini berfungsi untuk memberikan gelombang cahaya yang tepat ke area target.</p>\r\n\r\n<p>&middot; Sesi perawatan biasanya berlangsung sekitar 20-30 menit setiap sesinya, bergantung pada bagian tubuh dan luas area perawatan.</p>\r\n\r\n<p>&middot; Sebagian besar pasien dapat kembali bekerja dan beraktivitas seperti biasanya, segera setelah terapi selesai dilakukan (no downtime)</p>\r\n\r\n<p>Untuk mendapatkan hasil yang diinginkan, Anda mungkin perlu melakukan 3-6 sesi treatment. Perawatan tersebut harus berjarak sekitar 1 bulan untuk memberikan waktu pemulihan pada kulit Anda. Sementara, perawatan IPL pada rambut membutuhkan 6-12 sesi treatment.</p>\r\n',
 '<p><strong>Apa itu Face Lifting Radio Frequency?</strong></p>\r\n\r\n<p>Face Lifting Radio Frequency atau dikenal setrika wajah adalah treatment kulit yang menggunakan teknologi gelombang radio tanpa melalui proses luka (non ablatif).</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Face Lifting Radio Frequency</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Memperbaiki kerutan di dahi dan sekitar mata</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memperbaiki garis senyum yang dalam</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengencangkan kembali wajah yang kendur</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Face Lifting Radio Frequency?</strong></p>\r\n\r\n<p>Treatment Face Lifting Radio Frequency relatif aman dilakukan semua orang dengan warna kulit yang berbeda-beda. Anda dapat melakukan treatment ini jika menginginkan kulit wajah yang lebih kencang.</p>\r\n\r\n<p>Namun, treatment ini tidak dianjurkan bagi Anda yang sedang hamil atau sedang memakai alat pacu jantung. Gelombang frekuensi dikhawatirkan dapat mempengaruhi janin atau alat treatment tersebut.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Face Lifting Radio Frequency?</strong></p>\r\n\r\n<p>Pada tahapan pertama saat melakukan treatment ini, wajah Anda akan dibersihkan terlebih dahulu dari kotoran dan make up yang menempel.</p>\r\n\r\n<p>Setelah itu, area wajah akan di treatment dengan alat sejenis pen yang telah dialiri listrik.&nbsp;</p>\r\n\r\n<p>Bagian wajah yang akan di treatment seperti pipi, garis senyum, dagu, dan daerah wajah lainnya akan ditarik sesuai garis kulit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Berapa kali harus melakukan treatment Face Lifting Radio Frequency?</strong></p>\r\n\r\n<p>Treatment ini dilakukan setiap dua minggu sekali. Setelah melakukan Face Lifting Radio Frequency, sebaiknya Anda tidak melakukan treatment lain selama 3-7 hari.</p>\r\n\r\n<p>Namun, dalam rentan waktu tersebut Anda boleh melakulan facial untuk pengencangan atau suntik kolagen.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah ada efek samping dari Face Lifting Radio Frequency?</strong></p>\r\n\r\n<p>Face Lifting Radio Frequency diklaim memiliki efek samping yang lebih sedikit dari pada treatment lainnya. Bahkan, treatment ini sama sekali tidak menimbulkan efek samping yang berbahaya.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>Efek samping ringan seperti:</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Kulit kemerahan sementara (hanya beberapa jam)</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Beberapa orang sensitif terhadap risiko radiasi</p>\r\n\t</li>\r\n</ul>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/2I7YVNq0Zfw\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p><strong>Apa itu Bright Lift Aquapure for Rejuvenation?</strong></p>\r\n\r\n<p>Bright Lift Aquapure for Rejuvenation adalah salah satu treatment kecantikan tanpa operasi yang membuat kulit tampak lebih<br />\r\nmuda dan sehat.</p>\r\n\r\n<p><strong>Manfaat Bright Lift Aquapure for Rejuvenation</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Membersihkan sel kulit mati&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membuat kulit glowing dan kenyal</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghilangkan kerutan dan garis halus</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Melembapkan wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Meratakan warna kulit</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p><strong>Apakah saya dapat melakukan treatment Bright Lift Aquapure for Rejuvenation?</strong></p>\r\n\r\n<p>Treatment Bright Lift Aquapure for Rejuvenation sangat direkomendasikan bagi Anda yang ingin mencerahkan wajah khususnya dalam persiapan mengikuti acara penting seperti penikahan. Treatment ini dapat dilakukan oleh semua jenis kulit dan dapat meratakan warna kulit, menghilangkan minyak berlebih, dan menghilangkan kerutan.</p>\r\n\r\n<p><strong>Bagaimana cara kerja Bright Lift Aquapure for Rejuvenation?</strong></p>\r\n\r\n<p>Pada tahap pertama treatment Bright Lift Aquapure for Rejuvenation, wajah Anda beserta komedo akan dibersihkan. Setelah itu, wajah akan diberikan serum Aquapure oleh dokter dalam beberapa tahapan</p>\r\n\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/5cxA25dkkOE\r\n\r\n">\r\n\r\n</iframe>',
 '<p><strong>Apa itu Brightening Peeling?</strong></p>\r\n\r\n<p>Brightening Peeling adalah prosedur untuk mengangkat lapisan terluar kulit, agar lapisan tersebut dapat digantikan dengan lapisan kulit yang baru. Tujuannya adalah agar kulit menjadi tampak lebih halus, muda, dan cerah, bisa dilakukan di area wajah dan seluruh tubuh</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Brightening Peeling</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Menghilangkan kerutan dan garis halus</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghilangkan jerawat dan bekas jerawat</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Meratakan warna kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menyamarkan flek hitam&nbsp;</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Jenis-jenis Brightening Peeling</strong></p>\r\n\r\n<p>Treatment Brightening Peeling memiliki tiga jenis, yaitu Brightening Peeling ringan, Brightening Peeling sedang, dan Brightening Peeling dalam.&nbsp;</p>\r\n\r\n<p>Ketiga jenis tersebut disesuaikan dengan tingkat keluhan dan masalah pada kulit Anda. Untuk itu, Anda perlu berkonsultasi dahulu dengan dokter sebelum melakukan treatment tersebut.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Cara kerja Brightening Peeling</strong></p>\r\n\r\n<p>Sebelum melakukan tindakan, akan dilakukan pembersihan wajah terlebih dahulu. Setelah itu akan diberikan cairan peeling seperti asam glikolat, asam trikloroasetat, asam salisilat, asam laktat, atau asam karbol (fenol), ke area kulit Anda. Perawatan ini memberikan efek rangsangan pada muka yang dapat membiarkan kulit baru menggantikannya.</p>\r\n\r\n<p>Selama perawatan, kebanyakan orang merasakan sensasi yang menyengat. Hal ini wajar terjadi dan rasa sensasi tersebut dapat ditoleransi oleh Anda. Setelah itu, dilakukan kompres dingin pada kulit yang dapat mengurangi sensasi tersebut.</p>\r\n\r\n<p><strong>Setelah treatment Brightening Peeling</strong></p>\r\n\r\n<p>Setelah melakukan Brightening Peeling, biasanya kulit Anda akan mengalami kemerahan yang diikuti dengan pengelupasan kulit mati pada area perawatan. Hal ini akan dialami selama tiga hingga tujuh hari kedepan, kulit sementara lebih sensitif terhadap matahari, maka dari itu, gunakan tabir surya dengan SPH diatas 30 setiap harinya untuk memproteksi kondisi kulit Anda.</p>\r\n\r\n<p>Brightening Peeling sangat berguna untuk Anda yang memiliki kulit yang kusam dan menginginkan wajah yang cerah, atau lebih cenderung memiliki warna kulit yang tidak rata.</p>\r\n\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/NzvV1ELMXEQ\r\n\r\n">\r\n\r\n</iframe>',
 '<p><strong>Apa itu Anti Aging Gold Facial?</strong></p>\r\n\r\n<p>Anti Aging Gold Facial merupakan serangkaian treatment advanced facial yang dapat menstimulasi sel-sel kulit, memperlancar peredaran darah di wajah, dan menutrisi sel-sel kulit mati.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Anti Aging Gold Facial</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengurangi kerutan dan garis halus</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Melembapkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membantu meremajakan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghaluskan kulit</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan treatment Anti Aging Gold Facial ?</strong></p>\r\n\r\n<p>Treatment ini sangat cocok bagi Anda yang ingin menghilangkan tanda-tanda penuaan pada wajah, seperti kerutan dan garis halus tanpa rasa sakit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah treatment Anti Aging Gold Facial aman?</strong></p>\r\n\r\n<p>Treatment ini aman jika dilakukan oleh Dokter kulit profesional.</p>\r\n\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/_EmGEO4zQOg\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p><strong>Apa itu Acne Peeling?</strong></p>\r\n\r\n<p>Treatment Acne Peeling merupakan prosedur kecantikan tanpa injeksi (suntikan) yang dapat mengatasi kulit yang berjerawat.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Acne Peeling</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Menghilangkan jerawat dan bekasnya</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengatasi komedo</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengangkat sel kulit mati</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Meremajakan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membersihkan pori-pori&nbsp;</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Cara Kerja Acne Peeling</strong></p>\r\n\r\n<p>Prosedur Treatment Acne Peeling berlangsung selama 10-20 menit saja , pertama kulit wajah akan dibersihkan kemudian diolesi cairan kimia yang telah diformulasikan khusus. Setelah selesai, wajah akan dibersihkan kembali.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Kandungan dalam cairan kimia Acne Peeling</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p><em>Alpha Hydroxy Acid</em>, termasuk <em>Glycolid Acid</em> dan <em>Lactic Acid</em>. Keduanya merupakan senyawa organik dari buah-buahan dan tanaman lainnya yang dinilai mampu membentuk kolagen baru dan menghilangkan sel kulit mati.</p>\r\n\t</li>\r\n\t<li>\r\n\t<p><em>Salicylic Acid, </em>merupakan senyawa yang terdapat pada buah-buahan, dinilai mampu menghilangkan bekas jerawat, mengurangi kadar minyak, dan mempercepat proses regenerasi kulit.</p>\r\n\t</li>\r\n\t<li>\r\n\t<p><em>Mandelic Acid</em>, senyawa yang biasanya terdapat pada buah almon, berfungsi untuk mengurangi masalah pigmentasi pada kulit dan meratakan warna kulit.</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Acne Peeling</strong></p>\r\n\r\n<p>Treatment ini cocok bagi Anda yang ingin menghilangkan jerawat, baik jerawat ringan maupun beruntusan tanpa prosedur injeksi/ suntikan.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Efek samping Acne Peeling</strong></p>\r\n\r\n<p>Dalam beberapa kasus, Acne Peeling meninggalkan spot hiperpigmentasi karena jerawat yang sudah mengering. Namun, efek samping tersebut akan pulih dalam 3-5 hari setelahnya.</p>\r\n\r\n\r\n<iframe width="420" height="315"\r\n\r\nsrc="https://www.youtube.com/embed/gs-yKx_jSwI\r\n\r\n">\r\n\r\n</iframe>',
 '<p><strong>Apa itu Purifying Facial Treatment?</strong></p>\r\n\r\n<p>Purifying Facial merupakan salah satu treatment dengan memanfaatkan serum yang dirancang khusus untuk<em> acne / oily skin</em> (kulit berjerawat/ berminyak).</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Purifying Facial For Acne/ Oily Skin</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengurangi minyak berlebih pada kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi jerawat</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membersihkan kotoran yang menyumbat pori-pori</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi warna gelap di sekitar area mata</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengecilkan pori-pori</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan treatment Purifying&nbsp; Facial?</strong></p>\r\n\r\n<p>Treatment ini sangat cocok bagi Anda yang memiliki tipe kulit berjerawat maupun berminyak. Selain itu, treatment Purifying Facial membantu Anda mengatasi minyak berlebih pemicu jerawat.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah treatment Purifying Facial aman?</strong></p>\r\n\r\n<p>Treatment ini aman jika dilakukan oleh Dokter kulit profesional.</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/_EmGEO4zQOg\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p>Apa itu Bright Lift Aquapure for Oily and Acne Skin?</p>\r\n\r\n<p>Bright Lift Aquapure for Oily and Acne Skin adalah perawatan yang menggunakan serum khusus untuk mengatasi berbagai masalah kulit berjerawat dan berminyak.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat </strong>Bright Lift Aquapure for Oily and Acne Skin</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Membersihkan sel kulit mati dan kotoran yang menempel di kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membuat kulit wajah kenyal dan glowing</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Melembapkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi kerutan dan garis halus</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi jerawat dan bekasnya</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan treatment Bright Lift Aquapure for Oily and Acne Skin?</strong></p>\r\n\r\n<p>Treatment Bright Lift Aquapure for Oily and Acne Skin sangat cocok bagi Anda yang memiliki kulit berjerawat maupun berminyak.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Bright Lift Aquapure for Oily and Acne Skin?</strong></p>\r\n\r\n<p>Pertama kulit wajah pasien akan diberikan serum khusus. Setelah itu, akan dilakukan prosedur Electroporation untuk memastikan serum masuk dan menyerap ke dalam kulit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah prosedur Bright Lift Aquapure for Oily and Acne Skin aman dilakukan?</strong></p>\r\n\r\n<p>Prosedur Bright Lift Aquapure for Oily and Acne Skin aman jika dilakukan oleh dokter kulit. Anda dapat melakukan treatment ini di Estetika dr. Affandi. Selain itu, treatment ini bersifat non-invasif sehingga tidak menimbulkan rasa sakit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan treatment Bright Lift Aquapure for Oily and Acne Skin?</strong></p>\r\n\r\n<p>Treatment Bright Lift Aquapure for Oily and Acne Skin sangat cocok bagi Anda yang memiliki kulit berjerawat maupun berminyak Selain itu, treatment ini bersifat non-invasif sehingga tidak menimbulkan rasa sakit.</p>\r\n\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/5cxA25dkkOE\r\n\r\n">\r\n\r\n</iframe>',
 '<p>Intensive Glowing Infusion merupakan salah satu treatment untuk mencerahkan kulit dengan metode Infus. Kompenen utama yang digunakan adalah vitamin C, kolagen dan antioksidan.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>MANFAAT VITAMIN C</strong></p>\r\n\r\n<p>Vitamin C merupakan vitamin yang penting dalam proses pembentukan kolagen, yaitu satu jenis protein penyusun jaringan ikat pada kulit, sendi, pembuluh darah, mata, hati, dan berbagai ogan tubuh lain. Bagi kulit, pembentukan kolagen ini menyebabkan kulit tidak mudah keriput dan tetap elastis. Cukup banyak alasan penggunaan vitamin C untuk kecantikan, terutama untuk mencegah kerut-kerut tanda penuaan. Vitamin C juga memiliki efek mencegah pigmentasi atau pewarnaan kulit. Hal ini karena vitamin C merupakan antioksidan kuat yang dapat menangkal proses oksidasi yang diperlukan dalam pembentukan melanin (pigmen yang menyebabkan warna gelap pada kulit yang terpapar sinar matahari dalam jangka waktu lama).</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Tujuan Intensive Glowing Infusion</strong></p>\r\n\r\n<p>Tujuan Intensive Glowing Infusion adalah untuk mencegah kerusakan jaringan ikat penyokong kulit (kolagen) akibat efek radikal bebas (utamanya akibar sinar matahari), sehingga kulit akan terlindungi dari efek penuaan dini. Kandungan vitamin C juga akan mencerahkan kulit dengan mengubah pembentukan melanin (zat warna kulit) menjadi lebih terang. Sebelum melakukan treatment Intensive Glowing Infusion ada baiknya kita untuk mengetahui sebatas mana kulit bisa menjadi lebih cerah. Coba lihat kulit lengan atas bagian dalam, itulah warna asli kulit kita. Jadi, efek pencerahan maksimal juga sebatas warna kulit lengan atas tersebut. Jika hasilnya lebih putih dari warna kulit asli, waspadai bahan-bahan berbahaya di dalamnya. Satu hal yang perlu diingat, metode ini tidak efektif jika tidak diimbangi dengan perawatan kulit yang baik.</p>\r\n\r\nIntensive glowing Infusion\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/6djobEsP-ZE\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p><strong>Apa itu Brightening Injection?</strong></p>\r\n\r\n<p>Brightening Injection atau yang sering disebut Injeksi pencerahan adalah treatment injeksi yang bertujuan untuk mencerahkan kulit dengan komponen utama injeksi, yaitu vitamin c, kolagen, dan antioksidan.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Kelebihan Brightening Injection</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mencerahkan kulit secara keseluruhan</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memberikan nutrisi dan vitamin pada kulit&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencegah penuaan dini&nbsp;</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah Brightening Injection aman?</strong></p>\r\n\r\n<p>Treatment Brightening Injection ini aman jika dilakukan oleh Dokter kulit profesional, karena dosis vitamin yang akan diberikan ke Anda harus sesuai dengan kebutuhan dan kondisi kulit Anda.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Brightening Injection?</strong></p>\r\n\r\n<p>Siapa pun dapat melakukan treatment Brightening Injection kecuali pada ibu hamil dan anak-anak.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apa efek samping dari Brightening Injection?</strong></p>\r\n\r\n<p>Treatment ini tentunya memiliki efek samping. Jika terlalu banyak melakukan treatment ini dan tidak disesuaikan dengan kebutuhkan kulit maka akan timbul masalah kulit dan bekas suntikkan terasa nyeri. Maka itu konsultasikan kepada Dokter jikalau merasakan efek samping tersebut.</p>\r\n',
 '<p><strong>Apa itu Detox &amp; Deep Pore Cleanse Facial</strong></p>\r\n\r\n<p>Detox &amp; Deep Pore Cleanse Facial merupakan salah satu treatment dengan memanfaatkan serum yang dirancang khusus untuk<em> </em>kulit berminyak.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Detox &amp; Deep Pore Cleanse Facial?</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengurangi minyak berlebih pada kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi jerawat</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membersihkan kotoran yang menyumbat pori-pori</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengecilkan pori-pori</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan treatment Detox &amp; Deep Pore Cleanse Facial?</strong></p>\r\n\r\n<p>Treatment ini sangat cocok bagi Anda yang memiliki tipe kulit berjerawat maupun berminyak. Selain itu, treatment Detox &amp; Deep Pore Cleanse Facial membantu Anda mengatasi minyak berlebih pemicu jerawat.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah treatment Detox &amp; Deep Pore Cleanse Facial aman?</strong></p>\r\n\r\n<p>Treatment ini aman jika dilakukan oleh Dokter kulit profesional.</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/_EmGEO4zQOg\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p><strong>Apa itu Injeksi Jerawat?</strong></p>\r\n\r\n<p>Injeksi Jerawat atau dikenal dengan Suntik Jerawat adalah treatment yang dilakukan untuk mengatasi masalah jerawat pada kulit, mulai dari jerawat ringan hingga yang sudah parah.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Kelebihan Injeksi Jerawat</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengecilkan jerawat</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengatasi noda hitam pada wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Hasil terlihat dalam beberapa hari setelah injeksi dilakukan&nbsp;</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah Injeksi Jerawat aman?</strong></p>\r\n\r\n<p>Treatment Injeksi Jerawat tergolong aman jika dilakukan oleh dokter kulit, karena dosis yang akan diberikan harus sesuai dengan kondisi jerawat pasien.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Injeksi Jerawat?</strong></p>\r\n\r\n<p>Siapa pun dapat melakukan treatment Injeksi Jerawat, kecuali pada wanita hamil dan anak-anak / orang dengan riwayat penyakit ginjal, tekanan darah tinggi, diabetes, ganguan tiroid, tukak lambung, dan gagal jantung.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apa efek samping dari Injeksi Jerawat?</strong></p>\r\n\r\n<p>Treatment Injeksi Jerawat tentunya juga memiliki efek samping. Jika terlalu banyak dan terlalu sering menyuntikkan obat jerawat, bisa meninggalkan bekas luka pada area suntikan. Oleh karena itu, sebaiknya Anda melakukan konsultasi dengan Dokter terlebih dahulu jika kondisi tersebut terjadi.</p>\r\n',
 '<p><strong>Apa itu Crystal White Treatment?</strong></p>\r\n\r\n<p>Crystal White Treatment merupakan kombinasi dari dua perawatan, yaitu Micro Crystal Dermabrasion dan Brightening Peeling. Treatment ini memanfaatkan metode pengikisan sel kulit mati pada kulit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Crystal White Treatment</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengatasi jerawat dan bekasnya</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghaluskan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghilangkan komedo</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengecilkan pori-pori</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengatasi tanda-tanda penuaan</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Crystal White Treatment?</strong></p>\r\n\r\n<p>Crystal White Treatment sangat direkomendasikan bagi Anda yang ingin mengatasi berbagai permasalahan di wajah seperti melasma, tekstur kulit yang kasar, jerawat, photo-aging, dan sebagainya.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah Crystal White Treatment aman?</strong></p>\r\n\r\n<p>Crystal White Treatment aman dan tidak berbahaya jika dilakukan oleh Dokter kulit profesional.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Crystal White Treatment?</strong></p>\r\n\r\n<p>Dalam Crystal White Treatment, terdapat proses pengikisan dengan bubuk kristal/ diamond bertujuan untuk mengeluarkan sel kulit mati, sehingga dapat memproduksi sel kulit dan kolagen baru yang dapat mencerahkan dan mengencangkan kulit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>Selain itu, dalam proses treatment ini pasien akan diolesi oleh serum yang diformulasikan khusus untuk melembapkan dan mencerahkan kulit.</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/3h1r-iuZoaw\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p><strong>Apa itu Oxy Infusion?</strong></p>\r\n\r\n<p>Oxy Infusion adalah treatment pemberian oksigen secara maksimal yang akan diserap langsung oleh kulit melalui lapisan pori-pori sehingga kulit akan terlihat lebih terjaga kelembabannya.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Oxy Infusion</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mencerahkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghaluskan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membuat kulit wajah lebih lembab</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membersihkan wajah dari kotoran dan debu</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Oxy Infusion?</strong></p>\r\n\r\n<p>Treatment Oxy Infusion merupakan solusi untuk Anda yang ingin mengatasi kulit kusam dan membuat kulit wajah menjadi lebih lembab.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Oxy Infusion?</strong></p>\r\n\r\n<p>Pada tahap awal treatment Oxy Infusion menggunakan serum yang cepat meresap ke dalam kulit. Treatment memakan waktu sekitar 20 menit dan tidak membutuhkan waktu pemulihan khusus. Hasilnya pun dapat terlihat usai melakukan treatment Oxy Infusion yaitu kulit terlihat lebih lembab, sehat, dan cerah.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Efek Samping Oxy Infusion</strong></p>\r\n\r\n<p>Efek samping yang ditimbulkan oleh treatment ini tidak ada jadi tidak membutuhkan waktu pemulihan khusus. Hasil treatment ini bisa langsung terlihat dalam sekali perawatan karena menggunakan oksigen bertekanan tinggi.&nbsp;</p>\r\n',
 '<p><strong>Apa itu Iontophoresis?</strong></p>\r\n\r\n<p>Iontophoresis adalah treatment stimulasi listrik yang memasukan Pro Vitamin C melalui teknologi Iontophoresis.&nbsp; Dengan alat ini membantu menyuplai Vitamin C untuk bagian-bagian tubuh lain yang membutuhkannya.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Fungsi Vitamin C untuk Kulit</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mencerahkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghaluskan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengendalikan produksi minyak berlebih&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menetralisir radikal bebas</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencegah timbulnya flek hitam</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Iontophoresis?</strong></p>\r\n\r\n<p>Treatment Iontophoresis merupakan solusi bagi Anda yang ingin mencerahkan kulit dan menghaluskan kulit agar kulit tampak lebih muda tanpa kerutan dan garis-garis halus di wajah.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Iontophoresis?</strong></p>\r\n\r\n<p>Pada tahap awal perawatan Iontophoresis ini pasien akan dimasukan Pro-Vitamin melalui teknologi Iontophoresis dan juga menyuplai Vitamin C ke bagian-bagian tubuh yang membutuhkan.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Keunggulan Treatment Iontophoresis Vitamin C</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mencegah timbulnya flek</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan dan menghaluskan kulit&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Pengobatan Jerawat</p>\r\n\t</li>\r\n</ul>\r\n',
 '<p><strong>Apa itu Advance Whitening Facial Treatment?</strong></p>\r\n\r\n<p>Advance Whitening Facial merupakan salah satu treatment dengan memanfaatkan serum yang dirancang khusus untuk mencerahkan wajah.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Advance Whitening Facial</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Membersihkan kotoran yang menyumbat pori-pori</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Melembabkan wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi warna gelap di sekitar area mata</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan treatment Advance Whitening Facial?</strong></p>\r\n\r\n<p>Treatment ini sangat cocok bagi Anda yang memiliki wajah kusam &amp; ingin mencerahkan wajah. Selain itu, Treatment Advance Whitening Facial juga bisa mengakat sel kulit mati dan menghaluskan wajah.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah treatment Advance Whitening Facial aman?</strong></p>\r\n\r\n<p>Treatment ini aman jika dilakukan oleh Dokter kulit profesional.</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/_EmGEO4zQOg\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p><strong>Apa itu Laser eCO2?</strong></p>\r\n\r\n<p><strong>Treatment Laser eCO2 </strong>merupakan laser fractional yang menggunakan teknologi laser dan gas <em>Carbon Dioxide (CO2) </em>dengan tujuan membuat permukaan kulit menjadi rata, dengan cara menghantarkan energi cahaya yang berupa fraction atau portion sehingga membentuk microscopic column pada permukaan kulit.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Laser eCO2</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengatasi scar atau bekas jerawat yang berlubang</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Merangsang sel-sel kulit untuk memproduksi elastin dan serat kolagen.</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menyamarkan stretchmark, selulit, dan kulit berkerut</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menghilangkan pertumbuhan kulit jinak dan superfisial seperti kutil, milia, dan tahi lalat kecil</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mampu meremajakan kulit wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengecilkan pori-pori yang membesar</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Laser eCO2?</strong></p>\r\n\r\n<p>Laser eCO2 merupakan solusi bagi Anda yang memiliki permasalahan kulit seperti bekas jerawat, bopeng yang berlubang atau scar.&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Laser eCO2?</strong></p>\r\n\r\n<p>Laser eCO2 merupakan laser jenis ablasi. Jenis laser ini bekerja dengan mengangkat lapisan kulit terluar kulit (epidermis) dan memanaskan lapisan kulit di bawahnya (dermis). Proses tersebut akan merangsang pertumbuhan serat kolagen baru pada kulit. Serat ini dapat membantu pertumbuhan jaringan kulit baru yang lebih lembut, cerah, dan kencang.</p>\r\n\r\n<p><br />\r\n<strong>Efek samping Laser eCO2</strong></p>\r\n\r\n<p>Efek samping yang ditimbulkan dari treatment ini biasanya hanya seperti sensasi sedikit rasa panas. Untuk melindungi lapisan epidermis pasien, setelah melakukan treatment akan dilakukan pengompresan secara berkala menggunakan kasa steril dan larutan khusus yang dingin, tidak lupa juga diberikan salep antibiotik dan salep anti radang.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/W_kV9ORycrc\r\n">\r\n</iframe>\r\n',
 '<p><strong>Apa itu Premium Collagen Booster?</strong></p>\r\n\r\n<p><strong>Premium Collagen Booster </strong>adalah collagen stimulator filler generasi terbaru (filler yang menstimulasi produksi kolagen di kulit) yang<strong> </strong>berbahan dasar <strong>Poly D,L-Lactic-Acid (PDLLA)</strong>, yaitu senyawa polimer dari bahan dasar alami pati jagung atau kentang yang memililiki ketahanan lebih dari 2 tahun. PDLLA berfungsi menstimulasi sel-sel kulit untuk membentuk kolagen-nya sendiri dan jaringan baru sehingga dapat membantu mengembalikan keremajaan kulit. Bahan <strong>Poly D,L-Lactic-Acid (PDLLA) </strong>yang digunakan pada treatment Premium Collagen Booster<strong> </strong>sangat mudah diserap serta aman untuk tubuh karena telah mendapatkan sertifikasi dari <strong>US FDA&rsquo;s GRAS</strong> juga ijin registrasi dari Kementrian Kesehatan. Bahan PDLLA sendiri sudah digunakan secara meluas selama 30 tahun.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat Premium Collagen Booster</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Menambah volume untuk area &ndash; area yang cekung, seperti lipatan kulit dan kerutan</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membuat kulit lebih kencang</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Membuat kulit lebih kenyal</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Melembabkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memperbaiki tekstur kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menyamarkan garis &ndash; garis halus &amp; pori - pori</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Merangsang pertumbuhan kolagen pada kulit&nbsp;</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan Premium Collagen Booster?</strong></p>\r\n\r\n<p>Premium<strong> </strong>Collagen Booster sangat cocok untuk Anda yang:</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Ingin memiliki kulit yang lebih kencang dan segar pada area wajah, leher &amp; punggung tangan.</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memiliki garis &ndash; garis halus, kerutan - kerutan dan atau pori - pori besar di wajah.</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Ingin menyamarkan nasolabial folds yang dalam atau marionette lines.</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memiliki area &ndash; area yang mengalami loss volume misalnya seperti pada pelipis, pipi atau dahi yang cekung.</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Ingin menambah volume hidung &amp; dagu dengan hasil yang halus dan natural.</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menginginkan kulit yang sehat alami.</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Premium Collagen Booster?</strong></p>\r\n\r\n<p>Treatment Premium Collagen Booster dilakukan dengan cara menyuntikkan cairan <strong>Poly D,L-Lactic-Acid (PDLLA) </strong>pada area kulit wajah yang cekung seperti lipatan kulit dan kerutan untuk menstimulasi produksi kolagen di area tersebut sehingga mampu menambah volume kulit serta membuat kulit menjadi kencang, kenyal, lembab, cerah dan juga glowing. Hasil dapat terlihat 1 minggu setelah treatment dan bisa bertahan hingga lebih dari 24 bulan.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Efek samping setelah tindakan.</strong></p>\r\n\r\n<p>Setelah melakukan treatment ini terkadang muncul seperti pembengkakan dan kemerahan pada titik-titik area bekas suntikan. Namun, efek samping tersebut hanya sementara dan akan segera hilang dalam beberapa hari setelah treatment.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Treatment PREMIUM COLLAGEN BOOSTER tidak disarankan untuk Anda&nbsp; yang memiliki riwayat penyakit auto imun,</strong></p>\r\n\r\n<p><br />\r\n<br />\r\n&nbsp;</p>\r\n<iframe width="420" height="315"\r\n\r\nsrc="https://www.youtube.com/embed/Q5ee5_iS024\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p><strong>Apa itu DNA SALMON Injection?</strong></p>\r\n\r\n<p>DNA Salmon Injection merupakan salah satu metode perawatan kulit yang memanfaatkan extra Deoxyribonucleic Acid (DNA) ikan salmon. Treatment ini menggunakan metode injeksi (suntik), sehingga sering disebut sebagai injeksi DNA Salmon.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Jenis-Jenis DNA Salmon Injection:</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p><strong>DNA Salmon Skin Booster</strong></p>\r\n\t</li>\r\n\t<li>\r\n\t<p><strong>DNA Salmon Scar Recovery</strong></p>\r\n\t</li>\r\n\t<li>\r\n\t<p><strong>DNA Salmon Eye Firming</strong></p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Manfaat treatment DNA Salmon Injection</strong></p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p>Mengencangkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memperbaiki tekstur kulit</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menyamarkan lingkaran hitam di kelopak mata</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengecilkan pori-pori</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mengurangi kerutan di kelopak mata, sekitar mulut dan di leher</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Menyamarkan scar/bopeng</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Cara kerja treatment DNA Salmon Injection</strong></p>\r\n\r\n<p>Pada tahap pertama treatment DNA Salmon Injection, pasien akan dibersihkan wajahnya dengan <em>cleanser</em> lalu akan dilanjutkan dengan tahap anastesi. Anastesi ini berguna agar kulit tidak iritasi dan meradang. Setelah anastesi, DNA Salmon akan diinjeksi di seluruh bagian wajah. Tahap akhirnya adalah pengompresan wajah dan mengaplikasikan krim.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah treatment DNA Salmon Injection menimbulkan efek samping?</strong><strong>&nbsp;</strong></p>\r\n\r\n<p>Efek samping berupa tonjolan-tonjolan di kulit yang akan hilang dalam hitungan jam. Di samping itu, wajah akan terlihat kemerahan atau memar di area bekas suntikan. Karena itu pengaplikasian krim berguna untuk meredakan kemerahan juga memar bekas suntikan.</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/r_PKf0tDcS8">\r\n\r\n</iframe>',
 '<ul>\r\n\t<li>\r\n\t<p><strong>Apa itu Micro Crystal Dermabrasion?</strong><br />\r\n\tMicro Crystal Dermabrasion adalah salah satu metode peremajaan kulit secara fisik yang sangat sering dilakukan. Pada prosedur ini dilakukan proses pengikisan/abrasi bagian kulit terluar dengan menggunakan bahan bertekstur (dengan diamond atau mikrokristal) yang digosokkan secara lembut di atas permukaan kulit.<br />\r\n\t&nbsp;</p>\r\n\t</li>\r\n\t<li>\r\n\t<p><strong>Manfaat Micro Crystal Dermabrasion</strong></p>\r\n\t</li>\r\n</ul>\r\n\r\n<ol>\r\n\t<li>\r\n\t<p>Merangsang pembentukan serat kolagen</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memperbaiki teksturkulit agar lebih lembut</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memperhalus pori-pori kulit wajah</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Kulit menjadi kenyal</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Meningkatkan kadar ceramid (moisturizer alami kulit)</p>\r\n\t</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p><strong>Apakah saya dapat melakukan treatment Micro Crystal Dermabrasion?</strong><br />\r\n\tMicro Crystal Dermabrasion<strong> </strong>adalah sangat direkomendasikan bagi Anda yang mempunyai permasalahan kulit yang berkurang kekenyalannya, kulit yang bertekstur dan tidak rata.</p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p><strong>Bagaimana cara kerja Micro Crystal Dermabrasion?</strong></p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>Pada treatment Micro Crystal Dermabrasion, dikerjakan dengan menggunakan diamond/mikrokristal steril yang dikeluarkan dari selang kecil, diabrasikan pada kulit wajah dan pada waktu bersamaan alat vacuum akan menyedot kembali kristal yang dikeluarkan bersama kulit mati yang dilepaskan selama proses treatment.</p>\r\n\r\n\r\nMicro Crystal Dermabrasion\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/8az-lw5JSxI\r\n\r\n">\r\n\r\n</iframe>',
 '<ul>\r\n\t<li>\r\n\t<p><strong>Apa itu Soothing Facial?</strong></p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>Soothing Facial merupakan salah satu treatment dengan memanfaatkan serum yang dirancang khusus untuk kulit yang iritasi dan sensitif.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p><strong>Manfaat Soothing Facial?</strong></p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>- membersihkan kulit&nbsp;</p>\r\n\r\n<p>- melembabkan wajah</p>\r\n\r\n<p>- menenangkan kulit yang sensitif, kemerahan dan iritasi</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p><strong>Apakah saya dapat melakukan treatment Soothing Facial?</strong></p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>Treatment ini sangat cocok bagi Anda yang memiliki permasalahan kulit sensitif dan iritasi. Selain itu, Soothing Facial Treatment dapat membantu Anda menenangkan kulit yang kemerahan.</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<ul>\r\n\t<li>\r\n\t<p><strong>Apakah treatment Soothing Facial aman?</strong></p>\r\n\t</li>\r\n</ul>\r\n\r\n<p>Treatment ini aman karena dilakukan oleh Dokter kulit profesional.</p>\r\n\r\n<iframe width="420" height="315"\r\nsrc="https://www.youtube.com/embed/_EmGEO4zQOg\r\n\r\n">\r\n\r\n</iframe>\r\n',
 '<p><strong>Apa itu Ultimate Eye Peeling?</strong><br />\r\nUltimate eye peeling adalah treatment kecantikan untuk mencerahkan kulit di area bawah mata dan mengencangkan kulit area mata dengan menggunakan serum khusus.<br />\r\n&nbsp;</p>\r\n\r\n<p><strong>Manfaat Ultimate Eye Peeling</strong></p>\r\n\r\n<ol>\r\n\t<li>\r\n\t<p>Mengencangkan kulit di area mata</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Memperbaiki serta menyamarkan kerut-kerut halus di kulit area mata</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Mencerahkan dan memudarkan noda/flek di kulit area mata</p>\r\n\t</li>\r\n\t<li>\r\n\t<p>Merangsang pertumbuhan kolagen baru di kulit area mata</p>\r\n\t</li>\r\n</ol>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p><strong>Apakah saya dapat melakukan treatment Ultimate Eye Peeling?</strong><br />\r\nUltimate Eye Peeling adalah sangat direkomendasikan bagi Anda yang mempunyai permasalahan kulit di daerah area mata seperti kantung mata dan lingkaran hitam di kelopak mata. Treatment ini dapat dilakukan untuk semua jenis kulit dan dapat membantu mengencangkan kulit serta menyamarkan kerutan halus di area mata.<br />\r\n&nbsp;</p>\r\n\r\n<p><strong>Bagaimana cara kerja Ultimate Eye Peeling?</strong></p>\r\n\r\n<p>Pada treatment Ultimate Eye Peeling, wajah Anda akan dibersihkan terlebih dahulu untuk mengangkat kotoran yang ada di wajah. Selain itu, wajah akan diberikan serum yang cepat meresap ke dalam kulit oleh Dokter dalam beberapa tahap yang berfungsi untuk mengatasi permasalahan kulit di area mata lebih cepat terselesaikan.</p>\r\n\r\n\r\n<iframe width="420" height="315"\r\n\r\nsrc="https://www.youtube.com/embed/Y5zyECx_6YU\r\n\r\n">\r\n\r\n</iframe>\r\n\r\n'
]


In [3]:

preprocessed_documents = []
for text in documents:
    text = BeautifulSoup(text, "html.parser").get_text()
    text = text.lower()
    text = re.sub(r'[\n\t\r]', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    print(text)

    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('indonesian'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    preprocessed_document = " ".join(tokens)

    preprocessed_documents.append(preprocessed_document)



vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)

penuaan pada kulit wajah ditandai dengan munculnya keriput  vlek  kulit kering  dan pori pori membesar  hal ini terjadi pada wajah anda  anti aging solutions dari estetika dr affandi bisa jadi solusinya   rangkaian anti aging solutions  krim perawatan anti aging     jika digunakan secara teratur akan menyamarkan keriput halus dan menjaga kelembapan alami kulit  treatments     ultherapy by ulthera     sangat efektif untuk menyamarkan kerutan kerutan dalam di wajah  laser rejuvenation      mampu meremajakan kulit dan menghilangkan vlek vlek hitam  botox injection filler injection premium collagen booster threadlift   tanam benang skin booster   ultrahydro booster     melembapkan kulit dari dalam sekaligus menjaga kekencangan   elastisitas kulit  ipl  intense pulsed light      efektif mengatasi noda kulit di wajah  autoroller     dapat merangsang produksi kolagen dan memperbaiki tekstur kulit wajah  prp treatment     dapat menstimulasi produksi kolagen dan elastin pada kulit wajah  radio 

In [6]:
text = "jerawat"
text = BeautifulSoup(text, "html.parser").get_text()
text = text.lower()
text = re.sub(r'[\n\t\r]', ' ', text)
text = re.sub(r'[^a-zA-Z\s]', ' ', text)

tokens = word_tokenize(text)

stop_words = set(stopwords.words('indonesian'))
tokens = [token for token in tokens if token.lower() not in stop_words]

factory = StemmerFactory()
stemmer = factory.create_stemmer()
tokens = [stemmer.stem(token) for token in tokens]

preprocessed_document = " ".join(tokens)

query = preprocessed_document
query_vector = vectorizer.transform([query])
similarity_scores = cosine_similarity(query_vector, tfidf_matrix)

best_match_indices = heapq.nlargest(10, range(len(similarity_scores[0])), similarity_scores[0].__getitem__)
best_match_scores = [similarity_scores[0][index] for index in best_match_indices]

for index, score in zip(best_match_indices, best_match_scores):
    print(f"Dokumen {index}\t similarity: {score}")

Dokumen 0	 similarity: 0.3071455277779328
Dokumen 30	 similarity: 0.17716004258018828
Dokumen 7	 similarity: 0.1591195473126006
Dokumen 35	 similarity: 0.15301030080979222
Dokumen 39	 similarity: 0.13559068304107508
Dokumen 1	 similarity: 0.1343720155924498
Dokumen 2	 similarity: 0.11096914885768602
Dokumen 4	 similarity: 0.11063771276862222
Dokumen 26	 similarity: 0.0990688341422097
Dokumen 43	 similarity: 0.09677652198281507


In [4]:
def save_model(model, filepath):
    joblib.dump(model, filepath)

def load_model(filepath):
    return joblib.load(filepath)



In [ ]:
save_model(vectorizer, "tfidf_vectorizer.joblib")
save_model(tfidf_matrix, "tfidf_matrix.joblib")

In [5]:
loaded_vectorizer = load_model("tfidf_vectorizer.joblib")
loaded_tfidf_matrix = load_model("tfidf_matrix.joblib")

text = "penuaan pada kulit wajah ditandai dengan munculnya keriput"
text = BeautifulSoup(text, "html.parser").get_text()
text = text.lower()
text = re.sub(r'[\n\t\r]', ' ', text)
text = re.sub(r'[^a-zA-Z\s]', ' ', text)

tokens = word_tokenize(text)

stop_words = set(stopwords.words('indonesian'))
tokens = [token for token in tokens if token.lower() not in stop_words]

factory = StemmerFactory()
stemmer = factory.create_stemmer()
tokens = [stemmer.stem(token) for token in tokens]

preprocessed_document = " ".join(tokens)

query = preprocessed_document
query_vector = loaded_vectorizer.transform([query])
similarity_scores = cosine_similarity(query_vector, loaded_tfidf_matrix)

best_match_indices = heapq.nlargest(10, range(len(similarity_scores[0])), similarity_scores[0].__getitem__)
best_match_scores = [similarity_scores[0][index] for index in best_match_indices]

r = zip(best_match_indices, best_match_scores)
for index, score in r:
    print(f"Dokumen {index}\t similarity: {score}")


Dokumen 0	 similarity: 0.3071455277779328
Dokumen 30	 similarity: 0.17716004258018828
Dokumen 7	 similarity: 0.1591195473126006
Dokumen 35	 similarity: 0.15301030080979222
Dokumen 39	 similarity: 0.13559068304107508
Dokumen 1	 similarity: 0.1343720155924498
Dokumen 2	 similarity: 0.11096914885768602
Dokumen 4	 similarity: 0.11063771276862222
Dokumen 26	 similarity: 0.0990688341422097
Dokumen 43	 similarity: 0.09677652198281507
